# Cleanup and preparation of historical data for prediction models
This notebook will be used to clean up the historical data required for the prediction models to be used:

**Radiation from the day before the call** https://opendata.aemet.es/centrodedescargas/productosAEMET<br>
**Solar radiation of two days before the day of data collection** http://www.soda-pro.com/web-services/radiation/cams-radiation-service<br>
**Climate data for the five days prior to the call** https://openweathermap.org/api/one-call-api#history<br>
**Weather forecast for the 48 hours next to the call** https://openweathermap.org/api/one-call-api<br>


- [Preparation](#Preparation)<br>

### 1. [Weather data from 5 previous days](#Weather-data-from-5-previous-days)

### 2. [Final climate function](#Final-climate-function)

### 3. [Weather predictions for the next 2 days](#Weather-predictions-for-the-next-2-days)

### 4. [Final predictions function](#Final-predictions-function)

### 5. [Radiation data from the previous day](#Radiation-data-from-the-previous-day)

### 6. [Day before radiation final function](#Day-before-radiation-final-function)

### 7. [Radiation data from two days before](#Radiation-data-from-two-days-before)

### 8. [Radiation final function](#Radiation-final-function)

### 9. [Generation of the rows with the days in columns](#Generation-of-the-rows-with-the-days-in-columns)


**CAUTION:** It takes hours to run

## Preparation

The necessary libraries and datasets are imported

In [1]:
import numpy as np
import pandas as pd
import random
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import matplotlib.pyplot as plt
plt.style.use("seaborn")
from datetime import datetime, timedelta

The working directory is set

In [2]:
%cd /home/dsc/git/TFM/

/home/dsc/git/TFM


In [3]:
directorio = '/home/dsc/git/TFM/'

Only hours with sunlight are used

In [4]:
hora_ini = 4
hora_fin = 20

# Climate data from previous 5 days
<div style = "float:right"><a style="text-decoration:none" href = "#Cleanup-and-preparation-of-historical-data-for-prediction-models">

This data is obtained from the OpenWeather portal (thanks to a student license that allows making a large number of calls per day) (https://openweathermap.org/api/one-call-api#history). **Data in UTC.** The hourly weather data for the 5 days prior to the call is accessed. The fields obtained are:

- ``dt``: Time of historical data, Unix, UTC
- ``temp``: Temperature. Units: kelvin
- ``feels_like``:  Temperature. This accounts for the human perception of weather. Units: kelvin
- ``pressure``: Atmospheric pressure on the sea level, hPa
- ``humidity``: Humidity, %
- ``dew_point``: Atmospheric temperature below which water droplets begin to condense and dew can form. Units: kelvin
- ``clouds``: Cloudiness, %
- ``visibility``: Average visibility, metres
- ``wind_speed``: Wind speed. Wind speed. Units: m/s
- ``wind_gust``: Wind gust. Units: m/s
- ``wind_deg``: Wind direction, degrees (meteorological)
- ``rain``: Precipitation volume, mm
- ``snow``: Snow volume, mm
- ``weather``: Includes an id and other parameters

Hour X contains the data elapsed between X:00 and X:59

### Data is studied

One of the daily datasets is used first

In [17]:
# The csv of historical climate data is imported. Their fields have been explained in the notebook where they are obtained

df_clima = pd.read_csv('./data/Clima_OW/clima_ow_2021-04-05', sep=',')
df_clima.head()

,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,clouds,visibility,wind_speed,wind_deg,wind_gust,we,rain.1h
0,2021-04-04,00:00,2021-04-05,0252D,286.30,285.66,1014,82,283.30,0,10000.0,1.03,260,NaN,800,NaN
1,2021-04-04,01:00,2021-04-05,0252D,285.94,284.13,1014,82,282.95,0,10000.0,2.57,290,NaN,800,NaN
2,2021-04-04,02:00,2021-04-05,0252D,285.67,284.98,1014,87,283.57,0,10000.0,1.22,271,1.79,800,NaN
3,2021-04-04,03:00,2021-04-05,0252D,285.53,284.34,1014,82,282.55,0,10000.0,1.54,60,NaN,800,NaN
4,2021-04-04,04:00,2021-04-05,0252D,284.30,282.56,1014,93,283.21,0,10000.0,2.57,280,NaN,800,NaN


In [18]:
df_clima.shape

(41736, 16)

In [19]:
print("There are {} stations".format(len(np.unique(df_clima["estacion"]))))

There are 287 stations


The list of stations is loaded, which will serve as a list of example locations

In [20]:
df_estaciones = pd.read_csv(directorio + 'data/estaciones.csv')

In [21]:
df_estaciones.head()

,latitud,provincia,altitud,indicativo,nombre,longitud
0,413515N,BARCELONA,74,0252D,ARENYS DE MAR,023224E
1,411734N,BARCELONA,4,0076,BARCELONA AEROPUERTO,020412E
2,412506N,BARCELONA,408,0200E,"BARCELONA, FABRA",020727E
3,412326N,BARCELONA,6,0201D,BARCELONA,021200E
4,414312N,BARCELONA,291,0149X,MANRESA,015025E


Stations that are not in both data frames (stations and df_clima) are searched

In [22]:
estacion_quitar = []
# Stations in the station list
serie_indicativos = df_estaciones["indicativo"].unique().astype("str")
# Stations in the climate dataset
serie_estaciones = list(set(df_clima["estacion"].unique().astype("str")))

diferencia = len(serie_indicativos) - len(serie_estaciones)
print("The difference is: {}".format(diferencia))

# The callsigns of the stations in the list that are not in the dataset are saved
for i in range(0, len(serie_indicativos)):
    estacion = serie_indicativos[i]
    if estacion not in serie_estaciones:
        print("Number {} is not in the list, station {}".format(i, estacion))
        estacion_quitar.append(str(estacion))
        diferencia -= 1
        print("The difference is: {}".format(diferencia))
        
# The callsigns of the dataset stations that are not in the station list are saved
for i in range(0, len(serie_estaciones)):
    estacion = serie_estaciones[i]
    if (estacion not in list(serie_indicativos) and estacion not in estacion_quitar):
        print("Number {} is not in the list, station {}".format(i, estacion))
        estacion_quitar.append(str(estacion))
        diferencia += 1
        print("The difference is: {}".format(diferencia))
print(estacion_quitar)
print("The difference is: {}".format(diferencia))

The difference is: 0
Number 31 is not in the list, station 1249X
The difference is: -1
Number 210 is not in the list, station 1249I
The difference is: 0
['1249X', '1249I']
The difference is: 0


The list with the callsigns to be removed is displayed

In [23]:
estacion_quitar

['1249X', '1249I']

These stations are removed from the dataset

In [24]:
cont = 0
for i in range(0, len(df_clima["estacion"])):
    if str(df_clima["estacion"].loc[i]) in estacion_quitar:
        cont += 1
print("We have to remove {} rows from {}. {} will remain".format(cont, len(df_clima["estacion"]), len(df_clima["estacion"])-cont))

We have to remove 240 rows from 41736. 41496 will remain


A column is created that stores a *True* if the station in that row should be removed

In [25]:
estaciones_df = []

for i in range(0, len(df_clima["estacion"])):
    estacion = df_clima["estacion"].loc[i]
    if estacion in estacion_quitar:
        estaciones_df.append(True)
    else:
        estaciones_df.append(False)

In [26]:
df_clima.insert(len(df_clima.columns),"quitar",estaciones_df,True)

In [27]:
df_clima.head()

,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,clouds,visibility,wind_speed,wind_deg,wind_gust,we,rain.1h,quitar
0,2021-04-04,00:00,2021-04-05,0252D,286.30,285.66,1014,82,283.30,0,10000.0,1.03,260,NaN,800,NaN,False
1,2021-04-04,01:00,2021-04-05,0252D,285.94,284.13,1014,82,282.95,0,10000.0,2.57,290,NaN,800,NaN,False
2,2021-04-04,02:00,2021-04-05,0252D,285.67,284.98,1014,87,283.57,0,10000.0,1.22,271,1.79,800,NaN,False
3,2021-04-04,03:00,2021-04-05,0252D,285.53,284.34,1014,82,282.55,0,10000.0,1.54,60,NaN,800,NaN,False
4,2021-04-04,04:00,2021-04-05,0252D,284.30,282.56,1014,93,283.21,0,10000.0,2.57,280,NaN,800,NaN,False


In [28]:
df_clima.drop(df_clima[df_clima["quitar"] == True].index, inplace = True)

In [29]:
df_clima.reset_index(drop=True, inplace=True)
df_clima.head()

,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,clouds,visibility,wind_speed,wind_deg,wind_gust,we,rain.1h,quitar
0,2021-04-04,00:00,2021-04-05,0252D,286.30,285.66,1014,82,283.30,0,10000.0,1.03,260,NaN,800,NaN,False
1,2021-04-04,01:00,2021-04-05,0252D,285.94,284.13,1014,82,282.95,0,10000.0,2.57,290,NaN,800,NaN,False
2,2021-04-04,02:00,2021-04-05,0252D,285.67,284.98,1014,87,283.57,0,10000.0,1.22,271,1.79,800,NaN,False
3,2021-04-04,03:00,2021-04-05,0252D,285.53,284.34,1014,82,282.55,0,10000.0,1.54,60,NaN,800,NaN,False
4,2021-04-04,04:00,2021-04-05,0252D,284.30,282.56,1014,93,283.21,0,10000.0,2.57,280,NaN,800,NaN,False


In [30]:
print("Indeed, the new number of rows is {}".format(len(df_clima["estacion"])))

Indeed, the new number of rows is 41496


### Interesting columns are selected

We eliminate the column ``quitar``.
The rest of the columns are meteorological data that we will use to generate the model

In [31]:
df_clima.drop(['quitar'], axis=1, inplace = True)
df_clima.head()

,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,clouds,visibility,wind_speed,wind_deg,wind_gust,we,rain.1h
0,2021-04-04,00:00,2021-04-05,0252D,286.30,285.66,1014,82,283.30,0,10000.0,1.03,260,NaN,800,NaN
1,2021-04-04,01:00,2021-04-05,0252D,285.94,284.13,1014,82,282.95,0,10000.0,2.57,290,NaN,800,NaN
2,2021-04-04,02:00,2021-04-05,0252D,285.67,284.98,1014,87,283.57,0,10000.0,1.22,271,1.79,800,NaN
3,2021-04-04,03:00,2021-04-05,0252D,285.53,284.34,1014,82,282.55,0,10000.0,1.54,60,NaN,800,NaN
4,2021-04-04,04:00,2021-04-05,0252D,284.30,282.56,1014,93,283.21,0,10000.0,2.57,280,NaN,800,NaN


In [32]:
df_clima.shape

(41496, 16)

Using ``.info()`` NAs are seen

In [33]:
df_clima.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41496 entries, 0 to 41495
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              41496 non-null  object 
 1   hour              41496 non-null  object 
 2   fecha_prediccion  41496 non-null  object 
 3   estacion          41496 non-null  object 
 4   temp              41496 non-null  float64
 5   feels_like        41496 non-null  float64
 6   pressure          41496 non-null  int64  
 7   humidity          41496 non-null  int64  
 8   dew_point         41496 non-null  float64
 9   clouds            41496 non-null  int64  
 10  visibility        28978 non-null  float64
 11  wind_speed        41496 non-null  float64
 12  wind_deg          41496 non-null  int64  
 13  wind_gust         14129 non-null  float64
 14  we                41496 non-null  int64  
 15  rain.1h           1910 non-null   float64
dtypes: float64(7), int64(5), object(4)
memor

Only useful hours are saved

In [34]:
df_clima["hour"] = pd.to_numeric([np.nan if pd.isna(c) == True else c[:2] for c in df_clima["hour"]])

In [35]:
df_clima = df_clima[(df_clima["hour"] < hora_fin) & (df_clima["hour"] >= hora_ini)]
df_clima.reset_index(drop=True, inplace=True)
df_clima.head()

,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,clouds,visibility,wind_speed,wind_deg,wind_gust,we,rain.1h
0,2021-04-04,4,2021-04-05,0252D,284.30,282.56,1014,93,283.21,0,10000.0,2.57,280,NaN,800,NaN
1,2021-04-04,5,2021-04-05,0252D,284.59,282.57,1014,93,283.50,0,10000.0,3.09,290,NaN,800,NaN
2,2021-04-04,6,2021-04-05,0252D,284.41,283.88,1015,87,282.33,0,10000.0,0.51,0,NaN,800,NaN
3,2021-04-04,7,2021-04-05,0252D,284.99,284.61,1016,87,282.90,0,10000.0,0.51,0,NaN,800,NaN
4,2021-04-04,8,2021-04-05,0252D,286.41,286.17,1016,82,283.41,0,10000.0,0.51,0,NaN,800,NaN


In [36]:
df_clima.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27664 entries, 0 to 27663
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              27664 non-null  object 
 1   hour              27664 non-null  int64  
 2   fecha_prediccion  27664 non-null  object 
 3   estacion          27664 non-null  object 
 4   temp              27664 non-null  float64
 5   feels_like        27664 non-null  float64
 6   pressure          27664 non-null  int64  
 7   humidity          27664 non-null  int64  
 8   dew_point         27664 non-null  float64
 9   clouds            27664 non-null  int64  
 10  visibility        19627 non-null  float64
 11  wind_speed        27664 non-null  float64
 12  wind_deg          27664 non-null  int64  
 13  wind_gust         8970 non-null   float64
 14  we                27664 non-null  int64  
 15  rain.1h           1380 non-null   float64
dtypes: float64(7), int64(6), object(3)
memor

The complete dataset is generated. To do this, all the saved daily files are loaded and concatenated

In [37]:
fechas = []

# The first date is set (date of the first daily file)
now = datetime.now()
fecha_inicial = datetime(2021,4,5)

# The most current date is set
fecha_final = datetime(now.year,now.month,now.day)

# We get the list of days
rango_fechas = range((fecha_final - fecha_inicial).days + 1)
lista_fechas = [fechas.append("{}-{}-{}".format((fecha_inicial + timedelta(days = d)).year, "%02d" % (fecha_inicial + timedelta(days = d)).month, "%02d" % (fecha_inicial + timedelta(days = d)).day)) for d in rango_fechas] 

fechas

['2021-04-05',
 '2021-04-06',
 '2021-04-07',
 '2021-04-08',
 '2021-04-09',
 '2021-04-10',
 '2021-04-11',
 '2021-04-12',
 '2021-04-13',
 '2021-04-14',
 '2021-04-15',
 '2021-04-16',
 '2021-04-17',
 '2021-04-18',
 '2021-04-19',
 '2021-04-20',
 '2021-04-21',
 '2021-04-22',
 '2021-04-23',
 '2021-04-24',
 '2021-04-25',
 '2021-04-26',
 '2021-04-27',
 '2021-04-28',
 '2021-04-29',
 '2021-04-30',
 '2021-05-01',
 '2021-05-02',
 '2021-05-03',
 '2021-05-04',
 '2021-05-05',
 '2021-05-06',
 '2021-05-07',
 '2021-05-08',
 '2021-05-09',
 '2021-05-10',
 '2021-05-11',
 '2021-05-12',
 '2021-05-13',
 '2021-05-14',
 '2021-05-15',
 '2021-05-16',
 '2021-05-17',
 '2021-05-18',
 '2021-05-19',
 '2021-05-20',
 '2021-05-21',
 '2021-05-22',
 '2021-05-23',
 '2021-05-24',
 '2021-05-25',
 '2021-05-26',
 '2021-05-27',
 '2021-05-28',
 '2021-05-29',
 '2021-05-30',
 '2021-05-31',
 '2021-06-01',
 '2021-06-02',
 '2021-06-03',
 '2021-06-04',
 '2021-06-05',
 '2021-06-06',
 '2021-06-07',
 '2021-06-08',
 '2021-06-09',
 '2021-06-

In [38]:
df_clima_total = pd.DataFrame()


# For each day, the saved dataset is added
for date in fechas:
    
    try:
        df_clima_ow = pd.read_csv(directorio + 'data/Clima_OW/clima_ow_{}'.format(date))
        df_clima_total = df_clima_total.append(df_clima_ow, ignore_index = True)
    except:
        continue
    
print(df_clima_total.shape)
df_clima_total.head()

(2907096, 17)


,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,clouds,visibility,wind_speed,wind_deg,wind_gust,we,rain.1h,snow.1h
0,2021-04-04,00:00,2021-04-05,0252D,286.30,285.66,1014,82,283.30,0,10000.0,1.03,260,NaN,800,NaN,NaN
1,2021-04-04,01:00,2021-04-05,0252D,285.94,284.13,1014,82,282.95,0,10000.0,2.57,290,NaN,800,NaN,NaN
2,2021-04-04,02:00,2021-04-05,0252D,285.67,284.98,1014,87,283.57,0,10000.0,1.22,271,1.79,800,NaN,NaN
3,2021-04-04,03:00,2021-04-05,0252D,285.53,284.34,1014,82,282.55,0,10000.0,1.54,60,NaN,800,NaN,NaN
4,2021-04-04,04:00,2021-04-05,0252D,284.30,282.56,1014,93,283.21,0,10000.0,2.57,280,NaN,800,NaN,NaN


Sometimes a variable ``uvi`` appears. If it does, it is removed

In [ ]:
try:
    df_clima_total.drop(['uvi'], axis=1, inplace = True)
except:
    pass

### Management of NAs

The number of NAs per column and the % it represents of the total are shown

In [39]:
total = df_clima_total.isnull().sum().sort_values(ascending = False)
percent = (df_clima_total.isnull().sum() / df_clima_total.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
snow.1h,2906488,0.999791
rain.1h,2687220,0.924366
wind_gust,1932994,0.664923
visibility,864955,0.297532
pressure,0,0.000000
hour,0,0.000000
fecha_prediccion,0,0.000000
estacion,0,0.000000
temp,0,0.000000
feels_like,0,0.000000


There is 29% of Nas in ``visibility``. The missing values are replaced by the mean 

There is 66.5% Nas in ``wind_gust``. The missing values are replaced by the mean

In [40]:
df_clima_total.fillna({'visibility': df_clima_total["visibility"].mean(), 'wind_gust': df_clima_total["wind_gust"].mean()}, inplace = True)

There is 92% Nas in ``rain.1h``. This column will not be used in the model

There is 99% Nas in ``snow.1h``. This column will not be used in the model

In [41]:
df_clima_total.drop(['rain.1h'], axis=1, inplace = True)
df_clima_total.drop(['snow.1h'], axis=1, inplace = True)

In [42]:
df_clima_total.head()

,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,clouds,visibility,wind_speed,wind_deg,wind_gust,we
0,2021-04-04,00:00,2021-04-05,0252D,286.30,285.66,1014,82,283.30,0,10000.0,1.03,260,4.203233,800
1,2021-04-04,01:00,2021-04-05,0252D,285.94,284.13,1014,82,282.95,0,10000.0,2.57,290,4.203233,800
2,2021-04-04,02:00,2021-04-05,0252D,285.67,284.98,1014,87,283.57,0,10000.0,1.22,271,1.790000,800
3,2021-04-04,03:00,2021-04-05,0252D,285.53,284.34,1014,82,282.55,0,10000.0,1.54,60,4.203233,800
4,2021-04-04,04:00,2021-04-05,0252D,284.30,282.56,1014,93,283.21,0,10000.0,2.57,280,4.203233,800


In [43]:
df_clima_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2907096 entries, 0 to 2907095
Data columns (total 15 columns):
 #   Column            Dtype  
---  ------            -----  
 0   date              object 
 1   hour              object 
 2   fecha_prediccion  object 
 3   estacion          object 
 4   temp              float64
 5   feels_like        float64
 6   pressure          int64  
 7   humidity          int64  
 8   dew_point         float64
 9   clouds            int64  
 10  visibility        float64
 11  wind_speed        float64
 12  wind_deg          int64  
 13  wind_gust         float64
 14  we                int64  
dtypes: float64(6), int64(5), object(4)
memory usage: 332.7+ MB


Possible repeated rows are eliminated

In [44]:
df_clima_total = df_clima_total.drop_duplicates(['date', 'hour', "fecha_prediccion", "estacion"],
                        keep = 'first')
df_clima_total.reset_index(drop = True, inplace = True)
df_clima_total.head()

,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,clouds,visibility,wind_speed,wind_deg,wind_gust,we
0,2021-04-04,00:00,2021-04-05,0252D,286.30,285.66,1014,82,283.30,0,10000.0,1.03,260,4.203233,800
1,2021-04-04,01:00,2021-04-05,0252D,285.94,284.13,1014,82,282.95,0,10000.0,2.57,290,4.203233,800
2,2021-04-04,02:00,2021-04-05,0252D,285.67,284.98,1014,87,283.57,0,10000.0,1.22,271,1.790000,800
3,2021-04-04,03:00,2021-04-05,0252D,285.53,284.34,1014,82,282.55,0,10000.0,1.54,60,4.203233,800
4,2021-04-04,04:00,2021-04-05,0252D,284.30,282.56,1014,93,283.21,0,10000.0,2.57,280,4.203233,800


# Final climate function
<div style = "float:right"><a style="text-decoration:none" href = "#Cleanup-and-preparation-of-historical-data-for-prediction-models">

In [45]:
def clima_ow_clean(df_datos, df_puntos):
    
    # Stations that are not in both lists are deleted
    
    estacion_quitar = []
    # Stations in the station list
    serie_indicativos = df_puntos["indicativo"].unique().astype("str")
    # Stations in the climate dataset
    serie_estaciones = list(set(df_datos["estacion"].unique().astype("str")))

    diferencia = len(serie_indicativos) - len(serie_estaciones)

    # The callsigns of the stations in the list that are not in the dataset are saved
    for i in range(0, len(serie_indicativos)):
        estacion = serie_indicativos[i]
        if estacion not in serie_estaciones:
            estacion_quitar.append(str(estacion))
            diferencia -= 1
            
    # The callsigns of the dataset stations that are not in the station list are saved
    for i in range(0, len(serie_estaciones)):
        estacion = serie_estaciones[i]
        if (estacion not in list(serie_indicativos) and estacion not in estacion_quitar):
            estacion_quitar.append(str(estacion))
            diferencia += 1
            

    estaciones_df = []
    for i in range(0, len(df_datos["estacion"])):
        estacion = df_datos["estacion"].loc[i]
        if estacion in estacion_quitar:
            estaciones_df.append(True)
        else:
            estaciones_df.append(False)
            
    df_datos.insert(len(df_datos.columns), "quitar", estaciones_df, True)
    df_datos.drop(df_datos[df_datos["quitar"] == True].index, inplace = True)
    df_datos.reset_index(drop = True, inplace = True)
    
    # Unnecessary columns are removed
    
    try:
        df_datos.drop(['uvi'], axis=1, inplace = True)
    except:
        pass
    df_datos.drop(['quitar'], axis=1, inplace = True)
    df_datos.drop(['rain.1h'], axis=1, inplace = True)
    df_datos.drop(['snow.1h'], axis=1, inplace = True)
    
    # Columns are converted to the correct data types
    
    df_datos["hour"] = pd.to_numeric([np.nan if pd.isna(c) == True else c[:2] for c in df_datos["hour"]])
    df_datos = df_datos[(df_datos["hour"] < hora_fin) & (df_datos["hour"] >= hora_ini)]
    df_datos.reset_index(drop=True, inplace=True)
    
    # Na's are eliminated
    
    df_datos.fillna({'visibility': df_datos["visibility"].mean(), 'wind_gust': df_datos["wind_gust"].mean()}, inplace = True)
    
    # Repeated rows are removed
    
    df_datos = df_datos.drop_duplicates(['date', 'hour', "fecha_prediccion", "estacion"],
                        keep = 'first')
    df_datos.reset_index(drop = True, inplace = True)
    df_datos.head()

    # Clean file is saved
    nombre = './data/Historicos/historicos_climaticos_clean.csv'
    df_datos.to_csv(nombre, index = False)
    
    return df_datos

In [46]:
import numpy as np
import pandas as pd
import random
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import matplotlib.pyplot as plt
plt.style.use("seaborn")
from datetime import datetime, timedelta
hora_ini = 4
hora_fin = 20

# CSVs are read
fechas = []
now = datetime.now()
fecha_inicial = datetime(2021,4,5)
fecha_final = datetime(now.year,now.month,now.day)
rango_fechas = range((fecha_final - fecha_inicial).days + 1)
lista_fechas = [fechas.append("{}-{}-{}".format((fecha_inicial + timedelta(days = d)).year, "%02d" % (fecha_inicial + timedelta(days = d)).month, "%02d" % (fecha_inicial + timedelta(days = d)).day)) for d in rango_fechas] 

df_estaciones = pd.read_csv(directorio + 'data/estaciones.csv')

df_clima_total = pd.DataFrame()
for date in fechas:
    try:
        df_clima_ow = pd.read_csv(directorio + 'data/Clima_OW/clima_ow_{}'.format(date))
        df_clima_total = df_clima_total.append(df_clima_ow, ignore_index = True)
    except:
        continue

# The function is called
df_clean = clima_ow_clean(df_clima_total, df_estaciones)
    
df_clean.head()

/home/dsc/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,clouds,visibility,wind_speed,wind_deg,wind_gust,we
0,2021-04-04,4,2021-04-05,0252D,284.30,282.56,1014,93,283.21,0,10000.0,2.57,280,4.689019,800
1,2021-04-04,5,2021-04-05,0252D,284.59,282.57,1014,93,283.50,0,10000.0,3.09,290,4.689019,800
2,2021-04-04,6,2021-04-05,0252D,284.41,283.88,1015,87,282.33,0,10000.0,0.51,0,4.689019,800
3,2021-04-04,7,2021-04-05,0252D,284.99,284.61,1016,87,282.90,0,10000.0,0.51,0,4.689019,800
4,2021-04-04,8,2021-04-05,0252D,286.41,286.17,1016,82,283.41,0,10000.0,0.51,0,4.689019,800


In [47]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1870432 entries, 0 to 1870431
Data columns (total 15 columns):
 #   Column            Dtype  
---  ------            -----  
 0   date              object 
 1   hour              int64  
 2   fecha_prediccion  object 
 3   estacion          object 
 4   temp              float64
 5   feels_like        float64
 6   pressure          int64  
 7   humidity          int64  
 8   dew_point         float64
 9   clouds            int64  
 10  visibility        float64
 11  wind_speed        float64
 12  wind_deg          int64  
 13  wind_gust         float64
 14  we                int64  
dtypes: float64(6), int64(6), object(3)
memory usage: 214.1+ MB


### Sanity check

In [48]:
print('Different years: ', pd.to_datetime([year for year in pd.to_datetime(df_clean['date'], format='%Y/%m/%d')]).year.nunique())
print('Different months:', pd.to_datetime([year for year in pd.to_datetime(df_clean['date'], format='%Y/%m/%d')]).month.nunique())
print('There must be 31 different days:', pd.to_datetime([year for year in pd.to_datetime(df_clean['date'], format='%Y/%m/%d')]).day.nunique())
print('There should only be different hours within the filter hours:', df_clean['hour'].nunique())


print('Total longitude: ', len(df_clean["date"]))
print(df_clean.describe())

Different years:  1
Different months: 4
There must be 31 different days: 31
There should only be different hours within the filter hours: 16
Total longitude:  1870432
               hour          temp    feels_like      pressure      humidity  \
count  1.870432e+06  1.870432e+06  1.870432e+06  1.870432e+06  1.870432e+06   
mean   1.150000e+01  2.909718e+02 -7.737771e+12  1.014105e+03 -7.889833e+13   
std    4.609773e+00  5.925607e+00  2.645618e+15  9.826482e+00  2.697597e+16   
min    4.000000e+00  2.664400e+02 -9.082920e+17  6.820000e+02 -9.223372e+18   
25%    7.750000e+00  2.869100e+02  2.843200e+02  1.012000e+03  4.800000e+01   
50%    1.150000e+01  2.908700e+02  2.885100e+02  1.016000e+03  6.400000e+01   
75%    1.525000e+01  2.949700e+02  2.930300e+02  1.019000e+03  7.800000e+01   
max    1.900000e+01  3.111500e+02  3.091400e+02  1.047000e+03  1.000000e+02   

          dew_point        clouds    visibility    wind_speed      wind_deg  \
count  1.870432e+06  1.870432e+06  1.87043

# Weather predictions for the next 2 days
<div style = "float:right"><a style="text-decoration:none" href = "#Cleanup-and-preparation-of-historical-data-for-prediction-models">

This data is obtained from the OpenWeather portal (thanks to a student license that allows a large number of calls per day) (https://openweathermap.org/api/one-call-api). **Data in UTC.** The hourly weather forecast for the 2 days following the call is accessed. The fields obtained are:

- ``dt``: Time of the forecasted data, Unix, UTC
- ``temp``: Temperature. Units: kelvin
- ``feels_like``: Temperature. This accounts for the human perception of weather. Units: kelvin
- ``pressure``: Atmospheric pressure on the sea level, hPa
- ``humidity``: Humidity, %
- ``dew_point``: Atmospheric temperature (varying according to pressure and humidity) below which water droplets begin to condense and dew can form. Units: kelvin
- ``uvi``: UV index
- ``clouds``: Cloudiness, %
- ``visibility``: Average visibility, metres
- ``wind_speed``: Wind speed. Units: m/s
- ``wind_gust``: Wind gust. Units: m/s
- ``wind_deg``: Wind direction, degrees (meteorological)
- ``pop``: Probability of precipitation
- ``rain``: Rain volume for last hour, mm
- ``snow``: Snow volume for last hour, mm
- ``weather``: Includes an id and other parameters

Hour X contains the data elapsed between X:00 and X:59

### Data is studied

In [5]:
# The csv of climate predictions is imported

df_pred = pd.read_csv('./data/Pred_OW/pred_ow_2021-04-05', sep=',')
df_pred.head()

,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,pop,rain.1h,we,snow.1h
0,2021-04-04,22:00,2021-04-05,0252D,285.63,284.53,1017,61,278.32,0.0,7,10000,1.97,283,1.83,0.0,NaN,800,NaN
1,2021-04-04,23:00,2021-04-05,0252D,285.49,284.48,1018,65,279.11,0.0,10,10000,2.19,272,2.15,0.0,NaN,800,NaN
2,2021-04-05,00:00,2021-04-05,0252D,285.39,284.47,1018,69,279.88,0.0,18,10000,2.02,267,2.10,0.0,NaN,801,NaN
3,2021-04-05,01:00,2021-04-05,0252D,285.27,284.39,1018,71,280.18,0.0,47,10000,1.86,266,2.05,0.0,NaN,802,NaN
4,2021-04-05,02:00,2021-04-05,0252D,285.30,284.42,1017,71,280.21,0.0,54,10000,1.86,263,2.14,0.0,NaN,803,NaN


In [6]:
df_pred.shape

(22656, 19)

The station list is loaded

In [7]:
df_estaciones = pd.read_csv(directorio + 'data/estaciones.csv')

In [8]:
df_estaciones.head()

,latitud,provincia,altitud,indicativo,nombre,longitud
0,413515N,BARCELONA,74,0252D,ARENYS DE MAR,023224E
1,411734N,BARCELONA,4,0076,BARCELONA AEROPUERTO,020412E
2,412506N,BARCELONA,408,0200E,"BARCELONA, FABRA",020727E
3,412326N,BARCELONA,6,0201D,BARCELONA,021200E
4,414312N,BARCELONA,291,0149X,MANRESA,015025E


Stations that are not in both data frames (stations and df_pred) are searched

In [9]:
estacion_quitar = []
# Stations in the station list
serie_indicativos = df_estaciones["indicativo"].unique().astype("str")
# Stations in the climate dataset
serie_estaciones = list(set(df_pred["estacion"].unique().astype("str")))

diferencia = len(serie_indicativos) - len(serie_estaciones)
print("The difference is: {}".format(diferencia))

# Guardo los indicativos de las estaciones de la lista que no estan en el dataset
for i in range(0, len(serie_indicativos)):
    estacion = serie_indicativos[i]
    if estacion not in serie_estaciones:
        print("Number {} is not in the list, station {}".format(i, estacion))
        estacion_quitar.append(str(estacion))
        diferencia -= 1
        print("The difference is: {}".format(diferencia))
# Guardo los indicativos de las estaciones del dataset que no estan en la lista de estaciones
for i in range(0, len(serie_estaciones)):
    estacion = serie_estaciones[i]
    if (estacion not in list(serie_indicativos) and estacion not in estacion_quitar):
        print("Number {} is not in the list, station {}".format(i, estacion))
        estacion_quitar.append(str(estacion))
        diferencia += 1
        print("The difference is: {}".format(diferencia))
print(estacion_quitar)
print("The difference is: {}".format(diferencia))

The difference is: 0
Number 31 is not in the list, station 1249X
The difference is: -1
Number 124 is not in the list, station 1249I
The difference is: 0
['1249X', '1249I']
The difference is: 0


The list with the callsigns to be removed is displayed

In [10]:
estacion_quitar

['1249X', '1249I']

These stations are removed from the dataset

In [11]:
cont = 0
for i in range(0, len(df_pred["estacion"])):
    if str(df_pred["estacion"].loc[i]) in estacion_quitar:
        cont += 1
print("We have to remove {} rows from {}. {} will remain ".format(cont, len(df_pred["estacion"]), len(df_pred["estacion"])-cont))

We have to remove 96 rows from 22656. 22560 will remain 


A column that stores a *True* if the station in that row should be removed is created

In [12]:
estaciones_df = []

for i in range(0, len(df_pred["estacion"])):
    estacion = df_pred["estacion"].loc[i]
    if estacion in estacion_quitar:
        estaciones_df.append(True)
    else:
        estaciones_df.append(False)

In [13]:
df_pred.insert(len(df_pred.columns),"quitar",estaciones_df,True)

In [14]:
df_pred.head()

,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,pop,rain.1h,we,snow.1h,quitar
0,2021-04-04,22:00,2021-04-05,0252D,285.63,284.53,1017,61,278.32,0.0,7,10000,1.97,283,1.83,0.0,NaN,800,NaN,False
1,2021-04-04,23:00,2021-04-05,0252D,285.49,284.48,1018,65,279.11,0.0,10,10000,2.19,272,2.15,0.0,NaN,800,NaN,False
2,2021-04-05,00:00,2021-04-05,0252D,285.39,284.47,1018,69,279.88,0.0,18,10000,2.02,267,2.10,0.0,NaN,801,NaN,False
3,2021-04-05,01:00,2021-04-05,0252D,285.27,284.39,1018,71,280.18,0.0,47,10000,1.86,266,2.05,0.0,NaN,802,NaN,False
4,2021-04-05,02:00,2021-04-05,0252D,285.30,284.42,1017,71,280.21,0.0,54,10000,1.86,263,2.14,0.0,NaN,803,NaN,False


In [15]:
df_pred.drop(df_pred[df_pred["quitar"] == True].index, inplace = True)

In [16]:
df_pred.reset_index(drop=True, inplace=True)
df_pred.head()

,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,pop,rain.1h,we,snow.1h,quitar
0,2021-04-04,22:00,2021-04-05,0252D,285.63,284.53,1017,61,278.32,0.0,7,10000,1.97,283,1.83,0.0,NaN,800,NaN,False
1,2021-04-04,23:00,2021-04-05,0252D,285.49,284.48,1018,65,279.11,0.0,10,10000,2.19,272,2.15,0.0,NaN,800,NaN,False
2,2021-04-05,00:00,2021-04-05,0252D,285.39,284.47,1018,69,279.88,0.0,18,10000,2.02,267,2.10,0.0,NaN,801,NaN,False
3,2021-04-05,01:00,2021-04-05,0252D,285.27,284.39,1018,71,280.18,0.0,47,10000,1.86,266,2.05,0.0,NaN,802,NaN,False
4,2021-04-05,02:00,2021-04-05,0252D,285.30,284.42,1017,71,280.21,0.0,54,10000,1.86,263,2.14,0.0,NaN,803,NaN,False


In [17]:
print("Effectively, the new number of rows is {}".format(len(df_pred["estacion"])))

Effectively, the new number of rows is 22560


### Interesting columns are selected

The column ``quitar`` is eliminated.
The rest of the columns are meteorological data that we will use to generate the model.

In [18]:
df_pred.drop(['quitar'], axis=1, inplace = True)
df_pred.head()

,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,pop,rain.1h,we,snow.1h
0,2021-04-04,22:00,2021-04-05,0252D,285.63,284.53,1017,61,278.32,0.0,7,10000,1.97,283,1.83,0.0,NaN,800,NaN
1,2021-04-04,23:00,2021-04-05,0252D,285.49,284.48,1018,65,279.11,0.0,10,10000,2.19,272,2.15,0.0,NaN,800,NaN
2,2021-04-05,00:00,2021-04-05,0252D,285.39,284.47,1018,69,279.88,0.0,18,10000,2.02,267,2.10,0.0,NaN,801,NaN
3,2021-04-05,01:00,2021-04-05,0252D,285.27,284.39,1018,71,280.18,0.0,47,10000,1.86,266,2.05,0.0,NaN,802,NaN
4,2021-04-05,02:00,2021-04-05,0252D,285.30,284.42,1017,71,280.21,0.0,54,10000,1.86,263,2.14,0.0,NaN,803,NaN


In [19]:
df_pred.shape

(22560, 19)

Using ``.info()`` NAs are seen

In [20]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22560 entries, 0 to 22559
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              22560 non-null  object 
 1   hour              22560 non-null  object 
 2   fecha_prediccion  22560 non-null  object 
 3   estacion          22560 non-null  object 
 4   temp              22560 non-null  float64
 5   feels_like        22560 non-null  float64
 6   pressure          22560 non-null  int64  
 7   humidity          22560 non-null  int64  
 8   dew_point         22560 non-null  float64
 9   uvi               22560 non-null  float64
 10  clouds            22560 non-null  int64  
 11  visibility        22560 non-null  int64  
 12  wind_speed        22560 non-null  float64
 13  wind_deg          22560 non-null  int64  
 14  wind_gust         22560 non-null  float64
 15  pop               22560 non-null  float64
 16  rain.1h           521 non-null    float6

Only hours with sunlight are used

In [21]:
df_pred["hour"] = pd.to_numeric([np.nan if pd.isna(c) == True else c[:2] for c in df_pred["hour"]])

In [22]:
df_pred = df_pred[(df_pred["hour"] < hora_fin) & (df_pred["hour"] >= hora_ini)]
df_pred.reset_index(drop=True, inplace=True)
df_pred.head()

,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,pop,rain.1h,we,snow.1h
0,2021-04-05,4,2021-04-05,0252D,285.47,284.66,1017,73,280.28,0.00,63,10000,0.98,306,1.36,0.0,NaN,803,NaN
1,2021-04-05,5,2021-04-05,0252D,285.54,284.66,1017,70,279.91,0.00,65,10000,1.18,10,1.43,0.0,NaN,803,NaN
2,2021-04-05,6,2021-04-05,0252D,285.31,284.44,1018,71,279.75,0.11,61,10000,1.73,34,1.71,0.0,NaN,803,NaN
3,2021-04-05,7,2021-04-05,0252D,286.09,285.19,1017,67,279.64,0.55,16,10000,1.52,58,1.76,0.0,NaN,801,NaN
4,2021-04-05,8,2021-04-05,0252D,287.24,286.32,1017,62,279.58,1.46,10,10000,0.94,103,1.28,0.0,NaN,800,NaN


In [23]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15040 entries, 0 to 15039
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              15040 non-null  object 
 1   hour              15040 non-null  int64  
 2   fecha_prediccion  15040 non-null  object 
 3   estacion          15040 non-null  object 
 4   temp              15040 non-null  float64
 5   feels_like        15040 non-null  float64
 6   pressure          15040 non-null  int64  
 7   humidity          15040 non-null  int64  
 8   dew_point         15040 non-null  float64
 9   uvi               15040 non-null  float64
 10  clouds            15040 non-null  int64  
 11  visibility        15040 non-null  int64  
 12  wind_speed        15040 non-null  float64
 13  wind_deg          15040 non-null  int64  
 14  wind_gust         15040 non-null  float64
 15  pop               15040 non-null  float64
 16  rain.1h           431 non-null    float6

The complete dataset is generated, loading all the saved daily files and concatenating them

In [24]:
fechas = []

# The first date is set (date of the first daily file)
now = datetime.now()
fecha_inicial = datetime(2021,4,5)

# The most current date is set
fecha_final = datetime(now.year,now.month,now.day)

# The list of days is obtained
rango_fechas = range((fecha_final - fecha_inicial).days + 1)
lista_fechas = [fechas.append("{}-{}-{}".format((fecha_inicial + timedelta(days = d)).year, "%02d" % (fecha_inicial + timedelta(days = d)).month, "%02d" % (fecha_inicial + timedelta(days = d)).day)) for d in rango_fechas] 

fechas

['2021-04-05',
 '2021-04-06',
 '2021-04-07',
 '2021-04-08',
 '2021-04-09',
 '2021-04-10',
 '2021-04-11',
 '2021-04-12',
 '2021-04-13',
 '2021-04-14',
 '2021-04-15',
 '2021-04-16',
 '2021-04-17',
 '2021-04-18',
 '2021-04-19',
 '2021-04-20',
 '2021-04-21',
 '2021-04-22',
 '2021-04-23',
 '2021-04-24',
 '2021-04-25',
 '2021-04-26',
 '2021-04-27',
 '2021-04-28',
 '2021-04-29',
 '2021-04-30',
 '2021-05-01',
 '2021-05-02',
 '2021-05-03',
 '2021-05-04',
 '2021-05-05',
 '2021-05-06',
 '2021-05-07',
 '2021-05-08',
 '2021-05-09',
 '2021-05-10',
 '2021-05-11',
 '2021-05-12',
 '2021-05-13',
 '2021-05-14',
 '2021-05-15',
 '2021-05-16',
 '2021-05-17',
 '2021-05-18',
 '2021-05-19',
 '2021-05-20',
 '2021-05-21',
 '2021-05-22',
 '2021-05-23',
 '2021-05-24',
 '2021-05-25',
 '2021-05-26',
 '2021-05-27',
 '2021-05-28',
 '2021-05-29',
 '2021-05-30',
 '2021-05-31',
 '2021-06-01',
 '2021-06-02',
 '2021-06-03',
 '2021-06-04',
 '2021-06-05',
 '2021-06-06',
 '2021-06-07',
 '2021-06-08',
 '2021-06-09',
 '2021-06-

In [25]:
df_pred_total = pd.DataFrame()

# For each day, the saved dataset is added
for date in fechas:
    try:
        df_pred_ow = pd.read_csv(directorio + 'data/Pred_OW/pred_ow_{}'.format(date))
        df_pred_total = df_pred_total.append(df_pred_ow, ignore_index = True)
    except:
        continue
    
print(df_pred_total.shape)
df_pred_total.head()

(1186128, 19)


,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,pop,rain.1h,we,snow.1h
0,2021-04-04,22:00,2021-04-05,0252D,285.63,284.53,1017,61,278.32,0.0,7,10000,1.97,283,1.83,0.0,NaN,800,NaN
1,2021-04-04,23:00,2021-04-05,0252D,285.49,284.48,1018,65,279.11,0.0,10,10000,2.19,272,2.15,0.0,NaN,800,NaN
2,2021-04-05,00:00,2021-04-05,0252D,285.39,284.47,1018,69,279.88,0.0,18,10000,2.02,267,2.10,0.0,NaN,801,NaN
3,2021-04-05,01:00,2021-04-05,0252D,285.27,284.39,1018,71,280.18,0.0,47,10000,1.86,266,2.05,0.0,NaN,802,NaN
4,2021-04-05,02:00,2021-04-05,0252D,285.30,284.42,1017,71,280.21,0.0,54,10000,1.86,263,2.14,0.0,NaN,803,NaN


### Management of NAs

The number of NAs per column and the % it represents of the total are shown

In [26]:
total = df_pred_total.isnull().sum().sort_values(ascending = False)
percent = (df_pred_total.isnull().sum() / df_pred_total.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
snow.1h,1185735,0.999669
rain.1h,1055018,0.889464
dew_point,0,0.000000
hour,0,0.000000
fecha_prediccion,0,0.000000
estacion,0,0.000000
temp,0,0.000000
feels_like,0,0.000000
pressure,0,0.000000
humidity,0,0.000000


There is 88% Nas in ``rain.1h``. This column will not be used in the model

There is 99% Nas in ``snow.1h``. This column will not be used in the model

In [27]:
df_pred_total.drop(['rain.1h'], axis=1, inplace = True)
df_pred_total.drop(['snow.1h'], axis=1, inplace = True)

In [28]:
df_pred_total.head()

,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,pop,we
0,2021-04-04,22:00,2021-04-05,0252D,285.63,284.53,1017,61,278.32,0.0,7,10000,1.97,283,1.83,0.0,800
1,2021-04-04,23:00,2021-04-05,0252D,285.49,284.48,1018,65,279.11,0.0,10,10000,2.19,272,2.15,0.0,800
2,2021-04-05,00:00,2021-04-05,0252D,285.39,284.47,1018,69,279.88,0.0,18,10000,2.02,267,2.10,0.0,801
3,2021-04-05,01:00,2021-04-05,0252D,285.27,284.39,1018,71,280.18,0.0,47,10000,1.86,266,2.05,0.0,802
4,2021-04-05,02:00,2021-04-05,0252D,285.30,284.42,1017,71,280.21,0.0,54,10000,1.86,263,2.14,0.0,803


In [29]:
df_pred_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186128 entries, 0 to 1186127
Data columns (total 17 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   date              1186128 non-null  object 
 1   hour              1186128 non-null  object 
 2   fecha_prediccion  1186128 non-null  object 
 3   estacion          1186128 non-null  object 
 4   temp              1186128 non-null  float64
 5   feels_like        1186128 non-null  float64
 6   pressure          1186128 non-null  int64  
 7   humidity          1186128 non-null  int64  
 8   dew_point         1186128 non-null  float64
 9   uvi               1186128 non-null  float64
 10  clouds            1186128 non-null  int64  
 11  visibility        1186128 non-null  int64  
 12  wind_speed        1186128 non-null  float64
 13  wind_deg          1186128 non-null  int64  
 14  wind_gust         1186128 non-null  float64
 15  pop               1186128 non-null  float64
 16  

Possible repeated rows are eliminated

In [30]:
df_pred_total = df_pred_total.drop_duplicates(['date', 'hour', "fecha_prediccion", "estacion"],
                        keep = 'first')
df_pred_total.reset_index(drop = True, inplace = True)
df_pred_total.head()

,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,pop,we
0,2021-04-04,22:00,2021-04-05,0252D,285.63,284.53,1017,61,278.32,0.0,7,10000,1.97,283,1.83,0.0,800
1,2021-04-04,23:00,2021-04-05,0252D,285.49,284.48,1018,65,279.11,0.0,10,10000,2.19,272,2.15,0.0,800
2,2021-04-05,00:00,2021-04-05,0252D,285.39,284.47,1018,69,279.88,0.0,18,10000,2.02,267,2.10,0.0,801
3,2021-04-05,01:00,2021-04-05,0252D,285.27,284.39,1018,71,280.18,0.0,47,10000,1.86,266,2.05,0.0,802
4,2021-04-05,02:00,2021-04-05,0252D,285.30,284.42,1017,71,280.21,0.0,54,10000,1.86,263,2.14,0.0,803


# Final predictions function
<div style = "float:right"><a style="text-decoration:none" href = "#Cleanup-and-preparation-of-historical-data-for-prediction-models">

In [31]:
def pred_clean(df_datos, df_puntos):
    
    # Stations that are not in both dataset are deleted
    
    estacion_quitar = []
    # Stations in the station list
    serie_indicativos = df_puntos["indicativo"].unique().astype("str")
    # Stations in the climate dataset
    serie_estaciones = list(set(df_datos["estacion"].unique().astype("str")))

    diferencia = len(serie_indicativos) - len(serie_estaciones)

    # The callsigns of the stations in the list that are not in the dataset are saved
    for i in range(0, len(serie_indicativos)):
        estacion = serie_indicativos[i]
        if estacion not in serie_estaciones:
            estacion_quitar.append(str(estacion))
            diferencia -= 1
    # The callsigns of the dataset stations that are not in the station list are saved
    for i in range(0, len(serie_estaciones)):
        estacion = serie_estaciones[i]
        if (estacion not in list(serie_indicativos) and estacion not in estacion_quitar):
            estacion_quitar.append(str(estacion))
            diferencia += 1
    
    estaciones_df = []
    for i in range(0, len(df_datos["estacion"])):
        estacion = df_datos["estacion"].loc[i]
        if estacion in estacion_quitar:
            estaciones_df.append(True)
        else:
            estaciones_df.append(False)
    
    df_datos.insert(len(df_datos.columns), "quitar", estaciones_df, True)
    df_datos.drop(df_datos[df_datos["quitar"] == True].index, inplace = True)
    df_datos.reset_index(drop=True, inplace=True)

    # Unnecessary columns are removed
    
    df_datos.drop(['quitar'], axis=1, inplace = True)
    
    # Columns are converted to the correct data types
    
    df_datos["hour"] = pd.to_numeric([np.nan if pd.isna(c) == True else c[:2] for c in df_datos["hour"]])
    df_datos = df_datos[(df_datos["hour"] < hora_fin) & (df_datos["hour"] >= hora_ini)]
    df_datos.reset_index(drop=True, inplace=True)
    
    # Na's are eliminated
    
    df_datos.drop(['rain.1h'], axis=1, inplace = True)
    df_datos.drop(['snow.1h'], axis=1, inplace = True)
    
    # Possible repeated rows are eliminated
    
    df_datos = df_datos.drop_duplicates(['date', 'hour', "fecha_prediccion", "estacion"],
                        keep = 'first')
    df_datos.reset_index(drop = True, inplace = True)
    df_datos.head()

    # Clean file is saved
    nombre = './data/Historicos/predicciones_climaticas_clean.csv'
    df_datos.to_csv(nombre, index = False)
    
    return df_datos

In [32]:
import numpy as np
import pandas as pd
import random
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import matplotlib.pyplot as plt
plt.style.use("seaborn")
from datetime import datetime, timedelta
hora_ini = 4
hora_fin = 20

# CSVs are read
fechas = []
now = datetime.now()
fecha_inicial = datetime(2021,4,5)
fecha_final = datetime(now.year,now.month,now.day)
rango_fechas = range((fecha_final - fecha_inicial).days + 1)
lista_fechas = [fechas.append("{}-{}-{}".format((fecha_inicial + timedelta(days = d)).year, "%02d" % (fecha_inicial + timedelta(days = d)).month, "%02d" % (fecha_inicial + timedelta(days = d)).day)) for d in rango_fechas] 

df_estaciones = pd.read_csv(directorio + 'data/estaciones.csv')

df_pred_total = pd.DataFrame()
for date in fechas:
    try:
        df_pred_ow = pd.read_csv(directorio + 'data/Pred_OW/pred_ow_{}'.format(date))
        df_pred_total = df_pred_total.append(df_pred_ow, ignore_index = True)
    except:
        continue


# The function is called
df_clean = pred_clean(df_pred_total, df_estaciones)
    
df_clean.head()

/home/dsc/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,pop,we
0,2021-04-05,4,2021-04-05,0252D,285.47,284.66,1017,73,280.28,0.00,63,10000,0.98,306,1.36,0.0,803
1,2021-04-05,5,2021-04-05,0252D,285.54,284.66,1017,70,279.91,0.00,65,10000,1.18,10,1.43,0.0,803
2,2021-04-05,6,2021-04-05,0252D,285.31,284.44,1018,71,279.75,0.11,61,10000,1.73,34,1.71,0.0,803
3,2021-04-05,7,2021-04-05,0252D,286.09,285.19,1017,67,279.64,0.55,16,10000,1.52,58,1.76,0.0,801
4,2021-04-05,8,2021-04-05,0252D,287.24,286.32,1017,62,279.58,1.46,10,10000,0.94,103,1.28,0.0,800


In [33]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768344 entries, 0 to 768343
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   date              768344 non-null  object 
 1   hour              768344 non-null  int64  
 2   fecha_prediccion  768344 non-null  object 
 3   estacion          768344 non-null  object 
 4   temp              768344 non-null  float64
 5   feels_like        768344 non-null  float64
 6   pressure          768344 non-null  int64  
 7   humidity          768344 non-null  int64  
 8   dew_point         768344 non-null  float64
 9   uvi               768344 non-null  float64
 10  clouds            768344 non-null  int64  
 11  visibility        768344 non-null  int64  
 12  wind_speed        768344 non-null  float64
 13  wind_deg          768344 non-null  int64  
 14  wind_gust         768344 non-null  float64
 15  pop               768344 non-null  float64
 16  we                76

### Sanity check

In [34]:
print('Different years: ', pd.to_datetime([year for year in pd.to_datetime(df_clean['date'], format='%Y/%m/%d')]).year.nunique())
print('Different months:', pd.to_datetime([year for year in pd.to_datetime(df_clean['date'], format='%Y/%m/%d')]).month.nunique())
print('There must be 31 different days:', pd.to_datetime([year for year in pd.to_datetime(df_clean['date'], format='%Y/%m/%d')]).day.nunique())
print('There should only be different hours within the filter hours:', df_clean['hour'].nunique())

print('Total longitude: ', len(df_clean["date"]))
print(df_clean.describe())

Different years:  1
Different months: 4
There must be 31 different days: 31
There should only be different hours within the filter hours: 16
Total longitude:  768344
                hour           temp     feels_like       pressure  \
count  768344.000000  768344.000000  768344.000000  768344.000000   
mean       11.485741     290.768064     290.085467    1016.066060   
std         4.606912       6.262700       6.505929       5.123971   
min         4.000000     264.670000     259.790000     666.000000   
25%         7.000000     286.480000     285.680000    1013.000000   
50%        11.000000     290.780000     290.210000    1016.000000   
75%        15.000000     294.850000     294.550000    1019.000000   
max        19.000000     312.150000     313.200000    1042.000000   

            humidity      dew_point            uvi         clouds  \
count  768344.000000  768344.000000  768344.000000  768344.000000   
mean       60.768229     281.823348       2.874095      55.180424   
std  

# Radiation data from the previous day
<div style = "float:right"><a style="text-decoration:none" href = "#Cleanup-and-preparation-of-historical-data-for-prediction-models">

**These data are only available for the different radiation stations**

Hourly accumulated data (**TRUE SOLAR TIME**) of global, direct, diffuse and infrared radiation. These data are obtained from the AEMET Opendata portal (https://opendata.aemet.es/centrodedescargas/productosAEMET). The fields obtained for each day are:

- ``Estación``: Name of the station
- ``Indicativo``: Indicative Climatological Station
- ``Tipo``: Measured variable (Global/Diffuse/Direct/Erythematic UV/Infrared)
- ``GL/DF/DT``: Hourly radiation accumulated between: (indicated hour -1) and (indicated hour) between 5 and 20. True Solar Time. Variables: Global/Diffuse/Direct (10 * kJ/m²)
- ``UVER``: Semi-hourly radiation accumulated between: (hour: indicated minutes - 30 minutes and (hour: indicated minutes) between 4:30 and 20. True Solar Time. Variables: Erythematic Ultraviolet Radiation (J/m²)
- ``IR``: Hourly radiation accumulated between (indicated hour -1) and (indicated hour) between 1 and 24 True Solar Time. Variables: Infrared radiation (10 * kJ/m²)
- ...

Hour X contains the data elapsed between (X-1):00 and X:00

### Data is studied


In [35]:
# A csv of radiation data from the previous day is imported

df_rad_aemet = pd.read_csv('./data/Rad_AEMET/rad_aemet_2021-04-05', sep=',')
df_rad_aemet.head()

,fecha,Estación,Indicativo,Tipo,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,SUMA,Tipo.1,5.1,6.1,7.1,8.1,9.1,10.1,11.1,12.1,13.1,14.1,15.1,16.1,17.1,18.1,19.1,20.1,SUMA.1,Tipo.2,5.2,6.2,7.2,8.2,9.2,10.2,11.2,12.2,13.2,14.2,15.2,16.2,17.2,18.2,19.2,20.2,SUMA.2,Tipo.3,4.5,5.3,5.5,6.3,6.5,7.3,7.5,8.3,8.5,9.3,9.5,10.3,10.5,11.3,11.5,12.3,12.5,13.3,13.5,14.3,14.5,15.3,15.5,16.3,16.5,17.3,17.5,18.3,18.5,19.3,19.5,20.3,SUMA.3,Tipo.4,1,2,3,4,5.4,6.4,7.4,8.4,9.4,10.4,11.4,12.4,13.4,14.4,15.4,16.4,17.4,18.4,19.4,20.4,21,22,23,24,SUMA.4
0,04-04-21,A CORUÑA,1387,GL,0.0,3.0,47.0,118.0,189.0,247.0,287.0,308.0,307.0,285.0,243.0,185.0,115.0,46.0,3.0,0.0,2382.0,DF,0.0,2.0,16.0,25.0,29.0,32.0,34.0,36.0,34.0,34.0,35.0,31.0,25.0,15.0,2.0,0.0,348.0,DT,0.0,16.0,166.0,260.0,303.0,324.0,333.0,335.0,339.0,330.0,313.0,293.0,252.0,162.0,12.0,0.0,3437.0,UVB,0.0,0.0,0.0,2.0,9.0,20.0,38.0,62.0,91.0,125.0,161.0,195.0,231.0,262.0,272.0,291.0,291.0,266.0,248.0,226.0,194.0,155.0,127.0,93.0,61.0,37.0,19.0,7.0,1.0,0.0,0.0,0.0,3485.0,IR,96.0,96.0,97.0,96.0,96.0,96.0,96.0,96.0,98.0,100.0,103.0,104.0,106.0,107.0,105.0,106.0,106.0,104.0,101.0,99.0,98.0,98.0,98.0,NaN,NaN
1,04-04-21,ALBACETE,8178D,GL,0.0,3.0,41.0,116.0,190.0,255.0,294.0,156.0,269.0,164.0,238.0,195.0,130.0,46.0,2.0,0.0,2104.0,DF,0.0,2.0,25.0,43.0,55.0,73.0,102.0,111.0,134.0,115.0,61.0,46.0,45.0,26.0,2.0,0.0,843.0,DT,0.0,1.0,77.0,190.0,241.0,258.0,235.0,52.0,153.0,58.0,248.0,259.0,211.0,81.0,0.0,0.0,2065.0,UVB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IR,100.0,100.0,99.0,103.0,98.0,97.0,97.0,98.0,100.0,103.0,109.0,124.0,122.0,124.0,109.0,108.0,110.0,117.0,123.0,111.0,119.0,121.0,121.0,124.0,2637.0
2,04-04-21,ALMERÍA AEROPUERTO,6325O,GL,0.0,0.0,18.0,59.0,122.0,174.0,201.0,285.0,262.0,245.0,121.0,94.0,41.0,10.0,1.0,0.0,1632.0,DF,0.0,0.0,19.0,61.0,96.0,123.0,141.0,138.0,100.0,98.0,91.0,65.0,41.0,10.0,1.0,0.0,983.0,DT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UVB,0.0,0.0,0.0,0.0,1.0,5.0,16.0,28.0,50.0,61.0,84.0,170.0,159.0,182.0,214.0,289.0,202.0,270.0,228.0,162.0,78.0,106.0,79.0,39.0,30.0,17.0,7.0,2.0,0.0,0.0,0.0,0.0,2479.0,IR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,04-04-21,b'Badajoz',4478G,GL,0.0,1.0,22.0,120.0,192.0,259.0,301.0,322.0,319.0,294.0,252.0,193.0,119.0,43.0,2.0,0.0,2439.0,DF,0.0,1.0,21.0,55.0,84.0,61.0,60.0,62.0,67.0,70.0,61.0,51.0,40.0,22.0,2.0,0.0,656.0,DT,0.0,0.0,1.0,154.0,186.0,274.0,293.0,297.0,288.0,272.0,263.0,244.0,195.0,97.0,2.0,1.0,2566.0,UVB,0.0,0.0,0.0,0.0,2.0,9.0,22.0,44.0,70.0,100.0,143.0,183.0,219.0,249.0,271.0,284.0,283.0,271.0,248.0,217.0,182.0,144.0,106.0,72.0,44.0,23.0,10.0,3.0,0.0,0.0,0.0,0.0,3200.0,IR,113.0,111.0,116.0,124.0,127.0,130.0,123.0,106.0,108.0,105.0,108.0,111.0,114.0,116.0,117.0,117.0,117.0,116.0,115.0,114.0,112.0,112.0,112.0,112.0,2755.0
4,04-04-21,BARCELONA,0201D,GL,0.0,1.0,37.0,127.0,195.0,143.0,110.0,136.0,155.0,284.0,230.0,173.0,94.0,36.0,1.0,0.0,1723.0,DF,0.0,1.0,22.0,58.0,84.0,96.0,116.0,136.0,147.0,145.0,117.0,91.0,56.0,23.0,1.0,0.0,1095.0,DT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UVB,0.0,0.0,0.0,0.0,3.0,8.0,18.0,30.0,38.0,85.0,98.0,91.0,95.0,89.0,99.0,137.0,134.0,140.0,199.0,175.0,143.0,115.0,84.0,58.0,35.0,19.0,9.0,3.0,0.0,0.0,0.0,0.0,1905.0,IR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_rad_aemet.shape

(35, 117)

Cargo la lista de estaciones

In [37]:
df_estaciones_rad = pd.read_csv(directorio + 'data/estaciones_rad.csv')

In [38]:
df_estaciones_rad.head()

,Estación,indicativo,latitud,longitud
0,b'A Coru\xc3\xb1a',1387,432157N,082517W
1,b'Albacete',8178D,390020N,015144W
2,b'Almer\xc3\xada Aeropuerto',6325O,365047N,022125W
3,b'Badajoz',4478G,413800N,005256W
4,b'Barcelona',0201D,412326N,021200E


The stations that are not in both data frames (stations_rad and df_rad_aemet) are searched

In [39]:
estacion_quitar = []
# Stations in the station list
serie_indicativos = df_estaciones_rad["indicativo"].unique().astype("str")
# Stations in the climate dataset
serie_estaciones = list(set(df_rad_aemet["Indicativo"].unique().astype("str")))

diferencia = len(serie_indicativos) - len(serie_estaciones)
print("The difference is: {}".format(diferencia))

# The callsigns of the stations in the list that are not in the dataset are saved
for i in range(0, len(serie_indicativos)):
    estacion = serie_indicativos[i]
    if estacion not in serie_estaciones:
        print("The number {} is missing, station {}".format(i, estacion))
        estacion_quitar.append(str(estacion))
        diferencia -= 1
        print("The difference is: {}".format(diferencia))

# The callsigns of the dataset stations that are not in the station list are saved
for i in range(0, len(serie_estaciones)):
    estacion = serie_estaciones[i]
    if (estacion not in list(serie_indicativos) and estacion not in estacion_quitar):
        print("The number {} is missing, station {}".format(i, estacion))
        estacion_quitar.append(str(estacion))
        diferencia += 1
        print("The difference is: {}".format(diferencia))
print(estacion_quitar)
print("The difference is: {}".format(diferencia))

The difference is: 0
[]
The difference is: 0


The list with the callsigns to be removed is displayed

In [40]:
estacion_quitar

[]

These stations are removed from the dataset

In [41]:
cont = 0
for i in range(0, len(df_rad_aemet["Indicativo"])):
    if str(df_rad_aemet["Indicativo"].loc[i]) in estacion_quitar:
        cont += 1
print("We have to remove {} rows from {}. {} will remain".format(cont, len(df_rad_aemet["Indicativo"]), len(df_rad_aemet["Indicativo"])-cont))

We have to remove 0 rows from 35. 35 will remain


A column is created that stores a *True* if the station in that row should be removed

In [42]:
estaciones_df = []

for i in range(0, len(df_rad_aemet["Indicativo"])):
    estacion = df_rad_aemet["Indicativo"].loc[i]
    if estacion in estacion_quitar:
        estaciones_df.append(True)
    else:
        estaciones_df.append(False)

In [43]:
df_rad_aemet.insert(len(df_rad_aemet.columns), "quitar", estaciones_df, True)

In [44]:
df_rad_aemet.head()

,fecha,Estación,Indicativo,Tipo,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,SUMA,Tipo.1,5.1,6.1,7.1,8.1,9.1,10.1,11.1,12.1,13.1,14.1,15.1,16.1,17.1,18.1,19.1,20.1,SUMA.1,Tipo.2,5.2,6.2,7.2,8.2,9.2,10.2,11.2,12.2,13.2,14.2,15.2,16.2,17.2,18.2,19.2,20.2,SUMA.2,Tipo.3,4.5,5.3,5.5,6.3,6.5,7.3,7.5,8.3,8.5,9.3,9.5,10.3,10.5,11.3,11.5,12.3,12.5,13.3,13.5,14.3,14.5,15.3,15.5,16.3,16.5,17.3,17.5,18.3,18.5,19.3,19.5,20.3,SUMA.3,Tipo.4,1,2,3,4,5.4,6.4,7.4,8.4,9.4,10.4,11.4,12.4,13.4,14.4,15.4,16.4,17.4,18.4,19.4,20.4,21,22,23,24,SUMA.4,quitar
0,04-04-21,A CORUÑA,1387,GL,0.0,3.0,47.0,118.0,189.0,247.0,287.0,308.0,307.0,285.0,243.0,185.0,115.0,46.0,3.0,0.0,2382.0,DF,0.0,2.0,16.0,25.0,29.0,32.0,34.0,36.0,34.0,34.0,35.0,31.0,25.0,15.0,2.0,0.0,348.0,DT,0.0,16.0,166.0,260.0,303.0,324.0,333.0,335.0,339.0,330.0,313.0,293.0,252.0,162.0,12.0,0.0,3437.0,UVB,0.0,0.0,0.0,2.0,9.0,20.0,38.0,62.0,91.0,125.0,161.0,195.0,231.0,262.0,272.0,291.0,291.0,266.0,248.0,226.0,194.0,155.0,127.0,93.0,61.0,37.0,19.0,7.0,1.0,0.0,0.0,0.0,3485.0,IR,96.0,96.0,97.0,96.0,96.0,96.0,96.0,96.0,98.0,100.0,103.0,104.0,106.0,107.0,105.0,106.0,106.0,104.0,101.0,99.0,98.0,98.0,98.0,NaN,NaN,False
1,04-04-21,ALBACETE,8178D,GL,0.0,3.0,41.0,116.0,190.0,255.0,294.0,156.0,269.0,164.0,238.0,195.0,130.0,46.0,2.0,0.0,2104.0,DF,0.0,2.0,25.0,43.0,55.0,73.0,102.0,111.0,134.0,115.0,61.0,46.0,45.0,26.0,2.0,0.0,843.0,DT,0.0,1.0,77.0,190.0,241.0,258.0,235.0,52.0,153.0,58.0,248.0,259.0,211.0,81.0,0.0,0.0,2065.0,UVB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IR,100.0,100.0,99.0,103.0,98.0,97.0,97.0,98.0,100.0,103.0,109.0,124.0,122.0,124.0,109.0,108.0,110.0,117.0,123.0,111.0,119.0,121.0,121.0,124.0,2637.0,False
2,04-04-21,ALMERÍA AEROPUERTO,6325O,GL,0.0,0.0,18.0,59.0,122.0,174.0,201.0,285.0,262.0,245.0,121.0,94.0,41.0,10.0,1.0,0.0,1632.0,DF,0.0,0.0,19.0,61.0,96.0,123.0,141.0,138.0,100.0,98.0,91.0,65.0,41.0,10.0,1.0,0.0,983.0,DT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UVB,0.0,0.0,0.0,0.0,1.0,5.0,16.0,28.0,50.0,61.0,84.0,170.0,159.0,182.0,214.0,289.0,202.0,270.0,228.0,162.0,78.0,106.0,79.0,39.0,30.0,17.0,7.0,2.0,0.0,0.0,0.0,0.0,2479.0,IR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,04-04-21,b'Badajoz',4478G,GL,0.0,1.0,22.0,120.0,192.0,259.0,301.0,322.0,319.0,294.0,252.0,193.0,119.0,43.0,2.0,0.0,2439.0,DF,0.0,1.0,21.0,55.0,84.0,61.0,60.0,62.0,67.0,70.0,61.0,51.0,40.0,22.0,2.0,0.0,656.0,DT,0.0,0.0,1.0,154.0,186.0,274.0,293.0,297.0,288.0,272.0,263.0,244.0,195.0,97.0,2.0,1.0,2566.0,UVB,0.0,0.0,0.0,0.0,2.0,9.0,22.0,44.0,70.0,100.0,143.0,183.0,219.0,249.0,271.0,284.0,283.0,271.0,248.0,217.0,182.0,144.0,106.0,72.0,44.0,23.0,10.0,3.0,0.0,0.0,0.0,0.0,3200.0,IR,113.0,111.0,116.0,124.0,127.0,130.0,123.0,106.0,108.0,105.0,108.0,111.0,114.0,116.0,117.0,117.0,117.0,116.0,115.0,114.0,112.0,112.0,112.0,112.0,2755.0,False
4,04-04-21,BARCELONA,0201D,GL,0.0,1.0,37.0,127.0,195.0,143.0,110.0,136.0,155.0,284.0,230.0,173.0,94.0,36.0,1.0,0.0,1723.0,DF,0.0,1.0,22.0,58.0,84.0,96.0,116.0,136.0,147.0,145.0,117.0,91.0,56.0,23.0,1.0,0.0,1095.0,DT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UVB,0.0,0.0,0.0,0.0,3.0,8.0,18.0,30.0,38.0,85.0,98.0,91.0,95.0,89.0,99.0,137.0,134.0,140.0,199.0,175.0,143.0,115.0,84.0,58.0,35.0,19.0,9.0,3.0,0.0,0.0,0.0,0.0,1905.0,IR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [45]:
df_rad_aemet.drop(df_rad_aemet[df_rad_aemet["quitar"] == True].index, inplace = True)

In [46]:
df_rad_aemet.reset_index(drop=True, inplace=True)
df_rad_aemet.head()

,fecha,Estación,Indicativo,Tipo,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,SUMA,Tipo.1,5.1,6.1,7.1,8.1,9.1,10.1,11.1,12.1,13.1,14.1,15.1,16.1,17.1,18.1,19.1,20.1,SUMA.1,Tipo.2,5.2,6.2,7.2,8.2,9.2,10.2,11.2,12.2,13.2,14.2,15.2,16.2,17.2,18.2,19.2,20.2,SUMA.2,Tipo.3,4.5,5.3,5.5,6.3,6.5,7.3,7.5,8.3,8.5,9.3,9.5,10.3,10.5,11.3,11.5,12.3,12.5,13.3,13.5,14.3,14.5,15.3,15.5,16.3,16.5,17.3,17.5,18.3,18.5,19.3,19.5,20.3,SUMA.3,Tipo.4,1,2,3,4,5.4,6.4,7.4,8.4,9.4,10.4,11.4,12.4,13.4,14.4,15.4,16.4,17.4,18.4,19.4,20.4,21,22,23,24,SUMA.4,quitar
0,04-04-21,A CORUÑA,1387,GL,0.0,3.0,47.0,118.0,189.0,247.0,287.0,308.0,307.0,285.0,243.0,185.0,115.0,46.0,3.0,0.0,2382.0,DF,0.0,2.0,16.0,25.0,29.0,32.0,34.0,36.0,34.0,34.0,35.0,31.0,25.0,15.0,2.0,0.0,348.0,DT,0.0,16.0,166.0,260.0,303.0,324.0,333.0,335.0,339.0,330.0,313.0,293.0,252.0,162.0,12.0,0.0,3437.0,UVB,0.0,0.0,0.0,2.0,9.0,20.0,38.0,62.0,91.0,125.0,161.0,195.0,231.0,262.0,272.0,291.0,291.0,266.0,248.0,226.0,194.0,155.0,127.0,93.0,61.0,37.0,19.0,7.0,1.0,0.0,0.0,0.0,3485.0,IR,96.0,96.0,97.0,96.0,96.0,96.0,96.0,96.0,98.0,100.0,103.0,104.0,106.0,107.0,105.0,106.0,106.0,104.0,101.0,99.0,98.0,98.0,98.0,NaN,NaN,False
1,04-04-21,ALBACETE,8178D,GL,0.0,3.0,41.0,116.0,190.0,255.0,294.0,156.0,269.0,164.0,238.0,195.0,130.0,46.0,2.0,0.0,2104.0,DF,0.0,2.0,25.0,43.0,55.0,73.0,102.0,111.0,134.0,115.0,61.0,46.0,45.0,26.0,2.0,0.0,843.0,DT,0.0,1.0,77.0,190.0,241.0,258.0,235.0,52.0,153.0,58.0,248.0,259.0,211.0,81.0,0.0,0.0,2065.0,UVB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IR,100.0,100.0,99.0,103.0,98.0,97.0,97.0,98.0,100.0,103.0,109.0,124.0,122.0,124.0,109.0,108.0,110.0,117.0,123.0,111.0,119.0,121.0,121.0,124.0,2637.0,False
2,04-04-21,ALMERÍA AEROPUERTO,6325O,GL,0.0,0.0,18.0,59.0,122.0,174.0,201.0,285.0,262.0,245.0,121.0,94.0,41.0,10.0,1.0,0.0,1632.0,DF,0.0,0.0,19.0,61.0,96.0,123.0,141.0,138.0,100.0,98.0,91.0,65.0,41.0,10.0,1.0,0.0,983.0,DT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UVB,0.0,0.0,0.0,0.0,1.0,5.0,16.0,28.0,50.0,61.0,84.0,170.0,159.0,182.0,214.0,289.0,202.0,270.0,228.0,162.0,78.0,106.0,79.0,39.0,30.0,17.0,7.0,2.0,0.0,0.0,0.0,0.0,2479.0,IR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,04-04-21,b'Badajoz',4478G,GL,0.0,1.0,22.0,120.0,192.0,259.0,301.0,322.0,319.0,294.0,252.0,193.0,119.0,43.0,2.0,0.0,2439.0,DF,0.0,1.0,21.0,55.0,84.0,61.0,60.0,62.0,67.0,70.0,61.0,51.0,40.0,22.0,2.0,0.0,656.0,DT,0.0,0.0,1.0,154.0,186.0,274.0,293.0,297.0,288.0,272.0,263.0,244.0,195.0,97.0,2.0,1.0,2566.0,UVB,0.0,0.0,0.0,0.0,2.0,9.0,22.0,44.0,70.0,100.0,143.0,183.0,219.0,249.0,271.0,284.0,283.0,271.0,248.0,217.0,182.0,144.0,106.0,72.0,44.0,23.0,10.0,3.0,0.0,0.0,0.0,0.0,3200.0,IR,113.0,111.0,116.0,124.0,127.0,130.0,123.0,106.0,108.0,105.0,108.0,111.0,114.0,116.0,117.0,117.0,117.0,116.0,115.0,114.0,112.0,112.0,112.0,112.0,2755.0,False
4,04-04-21,BARCELONA,0201D,GL,0.0,1.0,37.0,127.0,195.0,143.0,110.0,136.0,155.0,284.0,230.0,173.0,94.0,36.0,1.0,0.0,1723.0,DF,0.0,1.0,22.0,58.0,84.0,96.0,116.0,136.0,147.0,145.0,117.0,91.0,56.0,23.0,1.0,0.0,1095.0,DT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UVB,0.0,0.0,0.0,0.0,3.0,8.0,18.0,30.0,38.0,85.0,98.0,91.0,95.0,89.0,99.0,137.0,134.0,140.0,199.0,175.0,143.0,115.0,84.0,58.0,35.0,19.0,9.0,3.0,0.0,0.0,0.0,0.0,1905.0,IR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [47]:
print("Indeed, the new number of rows is {}".format(len(df_rad_aemet["Indicativo"])))

Indeed, the new number of rows is 35


The complete dataset is generated, loading all the saved daily files and concatenating them

In [48]:
fechas = []

# The first date is set (date of the first daily file)
now = datetime.now()
fecha_inicial = datetime(2021,4,5)

# The most current date is set
fecha_final = datetime(now.year,now.month,now.day)

# We get the list of days
rango_fechas = range((fecha_final - fecha_inicial).days + 1)
lista_fechas = [fechas.append("{}-{}-{}".format((fecha_inicial + timedelta(days = d)).year, "%02d" % (fecha_inicial + timedelta(days = d)).month, "%02d" % (fecha_inicial + timedelta(days = d)).day)) for d in rango_fechas] 

fechas

['2021-04-05',
 '2021-04-06',
 '2021-04-07',
 '2021-04-08',
 '2021-04-09',
 '2021-04-10',
 '2021-04-11',
 '2021-04-12',
 '2021-04-13',
 '2021-04-14',
 '2021-04-15',
 '2021-04-16',
 '2021-04-17',
 '2021-04-18',
 '2021-04-19',
 '2021-04-20',
 '2021-04-21',
 '2021-04-22',
 '2021-04-23',
 '2021-04-24',
 '2021-04-25',
 '2021-04-26',
 '2021-04-27',
 '2021-04-28',
 '2021-04-29',
 '2021-04-30',
 '2021-05-01',
 '2021-05-02',
 '2021-05-03',
 '2021-05-04',
 '2021-05-05',
 '2021-05-06',
 '2021-05-07',
 '2021-05-08',
 '2021-05-09',
 '2021-05-10',
 '2021-05-11',
 '2021-05-12',
 '2021-05-13',
 '2021-05-14',
 '2021-05-15',
 '2021-05-16',
 '2021-05-17',
 '2021-05-18',
 '2021-05-19',
 '2021-05-20',
 '2021-05-21',
 '2021-05-22',
 '2021-05-23',
 '2021-05-24',
 '2021-05-25',
 '2021-05-26',
 '2021-05-27',
 '2021-05-28',
 '2021-05-29',
 '2021-05-30',
 '2021-05-31',
 '2021-06-01',
 '2021-06-02',
 '2021-06-03',
 '2021-06-04',
 '2021-06-05',
 '2021-06-06',
 '2021-06-07',
 '2021-06-08',
 '2021-06-09',
 '2021-06-

In [49]:
df_rad_aemet_total = pd.DataFrame()

# For each day, the saved dataset is added
for date in fechas:
    try:
        df_rad_aemet = pd.read_csv(directorio + 'data/Rad_AEMET/rad_aemet_{}'.format(date))
        df_rad_aemet["fecha"] = (pd.to_datetime(date)- timedelta(days = 1)).strftime('%Y-%m-%d')
        df_rad_aemet_total = df_rad_aemet_total.append(df_rad_aemet, ignore_index = True)
    except:
        continue
    
print(df_rad_aemet_total.shape)
df_rad_aemet_total.head()

(3115, 117)


,fecha,Estación,Indicativo,Tipo,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,SUMA,Tipo.1,5.1,6.1,7.1,8.1,9.1,10.1,11.1,12.1,13.1,14.1,15.1,16.1,17.1,18.1,19.1,20.1,SUMA.1,Tipo.2,5.2,6.2,7.2,8.2,9.2,10.2,11.2,12.2,13.2,14.2,15.2,16.2,17.2,18.2,19.2,20.2,SUMA.2,Tipo.3,4.5,5.3,5.5,6.3,6.5,7.3,7.5,8.3,8.5,9.3,9.5,10.3,10.5,11.3,11.5,12.3,12.5,13.3,13.5,14.3,14.5,15.3,15.5,16.3,16.5,17.3,17.5,18.3,18.5,19.3,19.5,20.3,SUMA.3,Tipo.4,1,2,3,4,5.4,6.4,7.4,8.4,9.4,10.4,11.4,12.4,13.4,14.4,15.4,16.4,17.4,18.4,19.4,20.4,21,22,23,24,SUMA.4
0,2021-04-04,A CORUÑA,1387,GL,0.0,3.0,47.0,118.0,189.0,247.0,287.0,308.0,307.0,285.0,243.0,185.0,115.0,46.0,3.0,0.0,2382.0,DF,0.0,2.0,16.0,25.0,29.0,32.0,34.0,36.0,34.0,34.0,35.0,31.0,25.0,15.0,2.0,0.0,348.0,DT,0.0,16.0,166.0,260.0,303.0,324.0,333.0,335.0,339.0,330.0,313.0,293.0,252.0,162.0,12.0,0.0,3437.0,UVB,0.0,0.0,0.0,2.0,9.0,20.0,38.0,62.0,91.0,125.0,161.0,195.0,231.0,262.0,272.0,291.0,291.0,266.0,248.0,226.0,194.0,155.0,127.0,93.0,61.0,37.0,19.0,7.0,1.0,0.0,0.0,0.0,3485.0,IR,96.0,96.0,97.0,96.0,96.0,96.0,96.0,96.0,98.0,100.0,103.0,104.0,106.0,107.0,105.0,106.0,106.0,104.0,101.0,99.0,98.0,98.0,98.0,NaN,NaN
1,2021-04-04,ALBACETE,8178D,GL,0.0,3.0,41.0,116.0,190.0,255.0,294.0,156.0,269.0,164.0,238.0,195.0,130.0,46.0,2.0,0.0,2104.0,DF,0.0,2.0,25.0,43.0,55.0,73.0,102.0,111.0,134.0,115.0,61.0,46.0,45.0,26.0,2.0,0.0,843.0,DT,0.0,1.0,77.0,190.0,241.0,258.0,235.0,52.0,153.0,58.0,248.0,259.0,211.0,81.0,0.0,0.0,2065.0,UVB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IR,100.0,100.0,99.0,103.0,98.0,97.0,97.0,98.0,100.0,103.0,109.0,124.0,122.0,124.0,109.0,108.0,110.0,117.0,123.0,111.0,119.0,121.0,121.0,124.0,2637.0
2,2021-04-04,ALMERÍA AEROPUERTO,6325O,GL,0.0,0.0,18.0,59.0,122.0,174.0,201.0,285.0,262.0,245.0,121.0,94.0,41.0,10.0,1.0,0.0,1632.0,DF,0.0,0.0,19.0,61.0,96.0,123.0,141.0,138.0,100.0,98.0,91.0,65.0,41.0,10.0,1.0,0.0,983.0,DT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UVB,0.0,0.0,0.0,0.0,1.0,5.0,16.0,28.0,50.0,61.0,84.0,170.0,159.0,182.0,214.0,289.0,202.0,270.0,228.0,162.0,78.0,106.0,79.0,39.0,30.0,17.0,7.0,2.0,0.0,0.0,0.0,0.0,2479.0,IR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-04-04,b'Badajoz',4478G,GL,0.0,1.0,22.0,120.0,192.0,259.0,301.0,322.0,319.0,294.0,252.0,193.0,119.0,43.0,2.0,0.0,2439.0,DF,0.0,1.0,21.0,55.0,84.0,61.0,60.0,62.0,67.0,70.0,61.0,51.0,40.0,22.0,2.0,0.0,656.0,DT,0.0,0.0,1.0,154.0,186.0,274.0,293.0,297.0,288.0,272.0,263.0,244.0,195.0,97.0,2.0,1.0,2566.0,UVB,0.0,0.0,0.0,0.0,2.0,9.0,22.0,44.0,70.0,100.0,143.0,183.0,219.0,249.0,271.0,284.0,283.0,271.0,248.0,217.0,182.0,144.0,106.0,72.0,44.0,23.0,10.0,3.0,0.0,0.0,0.0,0.0,3200.0,IR,113.0,111.0,116.0,124.0,127.0,130.0,123.0,106.0,108.0,105.0,108.0,111.0,114.0,116.0,117.0,117.0,117.0,116.0,115.0,114.0,112.0,112.0,112.0,112.0,2755.0
4,2021-04-04,BARCELONA,0201D,GL,0.0,1.0,37.0,127.0,195.0,143.0,110.0,136.0,155.0,284.0,230.0,173.0,94.0,36.0,1.0,0.0,1723.0,DF,0.0,1.0,22.0,58.0,84.0,96.0,116.0,136.0,147.0,145.0,117.0,91.0,56.0,23.0,1.0,0.0,1095.0,DT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UVB,0.0,0.0,0.0,0.0,3.0,8.0,18.0,30.0,38.0,85.0,98.0,91.0,95.0,89.0,99.0,137.0,134.0,140.0,199.0,175.0,143.0,115.0,84.0,58.0,35.0,19.0,9.0,3.0,0.0,0.0,0.0,0.0,1905.0,IR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As the name of the time corresponds to the time at which it ends, the useful hours are defined differently:

In [50]:
hora_ini = 5
hora_fin = 21
dif = int(int(hora_fin)-int(hora_ini))

The downloaded data has a structure such that each row corresponds to a day, the different columns being the hourly values of the different fields. Therefore, a dataset is generated in which each row corresponds to a time and each column to a field.

Thus, a dataset is generated where the rows correspond to an hour, for each day and station

In [51]:
df_rad_horas = pd.DataFrame(columns = ["fecha", "hora", "estacion", "indicativo", "GL", "DF", "DT", "UVB", "IR"])


for i, fila in df_rad_aemet_total.iterrows():
    
    for j in range(0, dif):
        
        # For each row of the new dataframe, the hourly values are saved, based on the name of each column of the old dataframe (called, for each variable, as the hour of the data)
        # In the case of the UVB variable, the values are by half hours, then the sum of both is obtained
        
        hora = 5+j
        col_gl = str(hora)
        col_df = str(hora) + ".1"
        col_dt = str(hora) + ".2"
        col_uvb = str(hora) + ".3"
        col_uvb_2 = str(hora-1) + ".5"
        col_ir = str(hora) + ".4"
        df_rad_horas = df_rad_horas.append({'fecha' : fila["fecha"], 'estacion' : fila["Estación"], 'indicativo' : fila["Indicativo"], 'GL' : fila[col_gl], 'DF' : fila[col_df], 'DT' : fila[col_dt], 'UVB' : (fila[col_uvb] + fila[col_uvb_2]), 'IR' : fila[col_ir], 'hora' : hora-1}, ignore_index = True)

df_rad_horas.head()

,fecha,hora,estacion,indicativo,GL,DF,DT,UVB,IR
0,2021-04-04,4,A CORUÑA,1387,0.0,0.0,0.0,0.0,96.0
1,2021-04-04,5,A CORUÑA,1387,3.0,2.0,16.0,2.0,96.0
2,2021-04-04,6,A CORUÑA,1387,47.0,16.0,166.0,29.0,96.0
3,2021-04-04,7,A CORUÑA,1387,118.0,25.0,260.0,100.0,96.0
4,2021-04-04,8,A CORUÑA,1387,189.0,29.0,303.0,216.0,98.0


### Interesting columns are selected

Direct and diffuse radiation columns, derived from global radiation, are eliminated

In [52]:
df_rad_horas.drop(['DF'], axis=1, inplace = True)
df_rad_horas.drop(['DT'], axis=1, inplace = True)

In [53]:
df_rad_horas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49840 entries, 0 to 49839
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fecha       49840 non-null  object 
 1   hora        49840 non-null  object 
 2   estacion    49840 non-null  object 
 3   indicativo  49840 non-null  object 
 4   GL          47899 non-null  float64
 5   UVB         35526 non-null  float64
 6   IR          31244 non-null  float64
dtypes: float64(3), object(4)
memory usage: 2.7+ MB


### Numeric columns are converted to the correct data type

In [54]:
df_rad_horas["hora"] = pd.to_numeric([np.nan if pd.isna(c) == True else int(c) for c in df_rad_horas["hora"]])

10 KJ/m² are converted to W/m², J/m² to W/m² and 10 KJ/m² to W/m²

In [55]:
df_rad_horas["GL"] = df_rad_horas["GL"] *10/3.6
df_rad_horas["UVB"] = df_rad_horas["UVB"] *1/(3.6*1000)
df_rad_horas["IR"] = df_rad_horas["IR"] *10/3.6

In [56]:
df_rad_horas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49840 entries, 0 to 49839
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fecha       49840 non-null  object 
 1   hora        49840 non-null  int64  
 2   estacion    49840 non-null  object 
 3   indicativo  49840 non-null  object 
 4   GL          47899 non-null  float64
 5   UVB         35526 non-null  float64
 6   IR          31244 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 2.7+ MB


In [57]:
df_rad_horas.head()

,fecha,hora,estacion,indicativo,GL,UVB,IR
0,2021-04-04,4,A CORUÑA,1387,0.000000,0.000000,266.666667
1,2021-04-04,5,A CORUÑA,1387,8.333333,0.000556,266.666667
2,2021-04-04,6,A CORUÑA,1387,130.555556,0.008056,266.666667
3,2021-04-04,7,A CORUÑA,1387,327.777778,0.027778,266.666667
4,2021-04-04,8,A CORUÑA,1387,525.000000,0.060000,272.222222


### Management of NAs

The number of NAs per column and the % it represents of the total are shown

In [58]:
total = df_rad_horas.isnull().sum().sort_values(ascending = False)
percent = (df_rad_horas.isnull().sum() / df_rad_horas.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
IR,18596,0.373114
UVB,14314,0.287199
GL,1941,0.038945
indicativo,0,0.000000
estacion,0,0.000000
hora,0,0.000000
fecha,0,0.000000


There are between 28 and 37% NAs in ``IR`` and ``UVB``. The missing values are replaced by the mean.

In [59]:
df_rad_horas.fillna({'IR': df_rad_horas["IR"].mean(), "UVB": df_rad_horas["UVB"].mean()}, inplace = True)

There are 3.8% NAs in ``GL``. Rows with no data are removed

In [60]:
df_rad_horas.dropna(inplace = True)
df_rad_horas.reset_index(drop=True, inplace=True)
df_rad_horas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47899 entries, 0 to 47898
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fecha       47899 non-null  object 
 1   hora        47899 non-null  int64  
 2   estacion    47899 non-null  object 
 3   indicativo  47899 non-null  object 
 4   GL          47899 non-null  float64
 5   UVB         47899 non-null  float64
 6   IR          47899 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 2.6+ MB


Possible repeated rows are eliminated

In [61]:
df_rad_horas = df_rad_horas.drop_duplicates(['fecha', 'hora', "indicativo"],
                        keep = 'first')
df_rad_horas.reset_index(drop = True, inplace = True)
df_rad_horas.head()

,fecha,hora,estacion,indicativo,GL,UVB,IR
0,2021-04-04,4,A CORUÑA,1387,0.000000,0.000000,266.666667
1,2021-04-04,5,A CORUÑA,1387,8.333333,0.000556,266.666667
2,2021-04-04,6,A CORUÑA,1387,130.555556,0.008056,266.666667
3,2021-04-04,7,A CORUÑA,1387,327.777778,0.027778,266.666667
4,2021-04-04,8,A CORUÑA,1387,525.000000,0.060000,272.222222


# Day before radiation final function
<div style = "float:right"><a style="text-decoration:none" href = "#Cleanup-and-preparation-of-historical-data-for-prediction-models">

In [62]:
def rad_aemet_clean(df_datos, df_puntos):
    
    # Stations that are not in both datasets are removed
    
    estacion_quitar = []
    # Stations in the station list
    serie_indicativos = df_puntos["indicativo"].unique().astype("str")
    # Stations in the station list
    serie_estaciones = list(set(df_datos["Indicativo"].unique().astype("str")))

    diferencia = len(serie_indicativos) - len(serie_estaciones)

    # The callsigns of the stations in the list that are not in the dataset are saved
    for i in range(0, len(serie_indicativos)):
        estacion = serie_indicativos[i]
        if estacion not in serie_estaciones:
            estacion_quitar.append(str(estacion))
            diferencia -= 1
            
    # The callsigns of the dataset stations that are not in the station list are saved
    for i in range(0, len(serie_estaciones)):
        estacion = serie_estaciones[i]
        if (estacion not in list(serie_indicativos) and estacion not in estacion_quitar):
            estacion_quitar.append(str(estacion))
            diferencia += 1
    
    estaciones_df = []

    for i in range(0, len(df_datos["Indicativo"])):
        estacion = df_datos["Indicativo"].loc[i]
        if estacion in estacion_quitar:
            estaciones_df.append(True)
        else:
            estaciones_df.append(False)

    df_datos.insert(len(df_datos.columns), "quitar", estaciones_df, True)
    df_datos.drop(df_datos[df_datos["quitar"] == True].index, inplace = True)
    df_datos.reset_index(drop=True, inplace=True)
    
    # The hourly dataset is generated
    
    dif = int(int(hora_fin)-int(hora_ini))
    df_rad_horas = pd.DataFrame(columns = ["fecha", "hora", "estacion", "indicativo", "GL", "DF", "DT", "UVB", "IR"])

    for i, fila in df_datos.iterrows():

        for j in range(0, dif):
            hora = 5+j
            col_gl = str(hora)
            col_df = str(hora) + ".1"
            col_dt = str(hora) + ".2"
            col_uvb = str(hora) + ".3"
            col_uvb_2 = str(hora-1) + ".5"
            col_ir = str(hora) + ".4"
            df_rad_horas = df_rad_horas.append({'fecha' : fila["fecha"], 'estacion' : fila["Estación"], 'indicativo' : fila["Indicativo"], 'GL' : fila[col_gl], 'DF' : fila[col_df], 'DT' : fila[col_dt], 'UVB' : (fila[col_uvb] + fila[col_uvb_2]), 'IR' : fila[col_ir], 'hora' : hora-1}, ignore_index = True)
    df_rad_horas.drop(['DF'], axis=1, inplace = True)
    df_rad_horas.drop(['DT'], axis=1, inplace = True)
    
    # Columns are converted to the correct data types
    
    df_rad_horas["hora"] = pd.to_numeric([np.nan if pd.isna(c) == True else int(c) for c in df_rad_horas["hora"]])
    df_rad_horas["GL"] = df_rad_horas["GL"] *10/3.6
    df_rad_horas["UVB"] = df_rad_horas["UVB"] *1/(3.6*1000)
    df_rad_horas["IR"] = df_rad_horas["IR"] *10/3.6
    
    # Na's are eliminated
    
    df_rad_horas.fillna({'IR': df_rad_horas["IR"].mean(), "UVB": df_rad_horas["UVB"].mean()}, inplace = True)
    df_rad_horas.dropna(inplace = True)
    df_rad_horas.reset_index(drop=True, inplace=True)
    
    # Possible repeated rows are eliminated
    
    df_rad_horas = df_rad_horas.drop_duplicates(['fecha', 'hora', "indicativo"], keep = 'first')
    df_rad_horas.reset_index(drop = True, inplace = True)
    df_rad_horas.head()

    # Clean file is saved
    nombre = './data/Historicos/rad_aemet_clean.csv'
    df_rad_horas.to_csv(nombre, index = False)
    
    return df_rad_horas

In [63]:
import numpy as np
import pandas as pd
import random
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import matplotlib.pyplot as plt
plt.style.use("seaborn")
from datetime import datetime, timedelta
hora_ini = 5
hora_fin = 21

# CSVs are read
fechas = []
now = datetime.now()
fecha_inicial = datetime(2021,4,5)
fecha_final = datetime(now.year,now.month,now.day)
rango_fechas = range((fecha_final - fecha_inicial).days + 1)
lista_fechas = [fechas.append("{}-{}-{}".format((fecha_inicial + timedelta(days = d)).year, "%02d" % (fecha_inicial + timedelta(days = d)).month, "%02d" % (fecha_inicial + timedelta(days = d)).day)) for d in rango_fechas] 

df_rad_aemet_total = pd.DataFrame()

for date in fechas:
    try:
        df_rad_aemet = pd.read_csv(directorio + 'data/Rad_AEMET/rad_aemet_{}'.format(date))
        df_rad_aemet["fecha"] = (pd.to_datetime(date)- timedelta(days = 1)).strftime('%Y-%m-%d')
        df_rad_aemet_total = df_rad_aemet_total.append(df_rad_aemet, ignore_index = True)
    except:
        continue

df_estaciones_rad = pd.read_csv(directorio + 'data/estaciones_rad.csv')

# The function is called
df_clean = rad_aemet_clean(df_rad_aemet_total, df_estaciones_rad)
    
df_clean.head()

,fecha,hora,estacion,indicativo,GL,UVB,IR
0,2021-04-04,4,A CORUÑA,1387,0.000000,0.000000,266.666667
1,2021-04-04,5,A CORUÑA,1387,8.333333,0.000556,266.666667
2,2021-04-04,6,A CORUÑA,1387,130.555556,0.008056,266.666667
3,2021-04-04,7,A CORUÑA,1387,327.777778,0.027778,266.666667
4,2021-04-04,8,A CORUÑA,1387,525.000000,0.060000,272.222222


In [64]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45228 entries, 0 to 45227
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fecha       45228 non-null  object 
 1   hora        45228 non-null  int64  
 2   estacion    45228 non-null  object 
 3   indicativo  45228 non-null  object 
 4   GL          45228 non-null  float64
 5   UVB         45228 non-null  float64
 6   IR          45228 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 2.4+ MB


### Sanity check

In [65]:
print('Different years: ', pd.to_datetime([year for year in pd.to_datetime(df_clean['fecha'])]).year.nunique())
print('Different months:', pd.to_datetime([year for year in pd.to_datetime(df_clean['fecha'])]).month.nunique())
print('Different days:', pd.to_datetime([year for year in pd.to_datetime(df_clean['fecha'])]).day.nunique())

print('There should only be different hours within the filtering hours:', df_clean['hora'].nunique())

Different years:  1
Different months: 3
Different days: 31
There should only be different hours within the filtering hours: 16


# Radiation data from two days before
<div style = "float:right"><a style="text-decoration:none" href = "#Cleanup-and-preparation-of-historical-data-for-prediction-models">

These data are obtained from the CAMS Radiation Service portal of the European Union (http://www.soda-pro.com/web-services/radiation/cams-radiation-service). **In UTC hour.** Provide radiation for any date up to 2 days before the call (3 day delay). The fields obtained for each day are:

- ``Observation period``: Beginning/end of the time period with the format "yyyy-mm-ddTHH:MM:SS.S/yyyy-mm-ddTHH:MM:SS.S"
- ``TOA``: Irradiation on horizontal plane at the top of atmosphere (Wh/m2) computed from Solar Geometry 2
- ``Clear sky GHI``: Clear sky global irradiation on horizontal plane at ground level (Wh/m2)
- ``Clear sky BHI``: Clear sky beam irradiation on horizontal plane at ground level (Wh/m2)
- ``Clear sky DHI``: Clear sky diffuse irradiation on horizontal plane at ground level (Wh/m2)
- ``Clear sky BNI``: Clear sky beam irradiation on mobile plane following the sun at normal incidence (Wh/m2)
- ``GHI``: Global irradiation on horizontal plane at ground level (Wh/m2)
- ``BHI``: Beam irradiation on horizontal plane at ground level (Wh/m2)
- ``DHI``: Diffuse irradiation on horizontal plane at ground level (Wh/m2)
- ``BNI``: Beam irradiation on mobile plane following the sun at normal incidence (Wh/m2)
- ``Reliability``: Proportion of reliable data in the summarization (0-1)

Hour X contains the data elapsed between X:00 and X:59

### Data is studied

It is first tested with a daily file

In [66]:
# The radiation data csv from two days ago is imported

df_soda = pd.read_csv('./data/Rad_SODA/rad_soda_2021-04-19', sep=',')
df_soda.head()

,dateBegins,dateEnds,toa,cs_ghi,cs_bhi,cs_dhi,cs_bni,ghi,bhi,dhi,bni,reliability,estacion
0,2021-04-16T00:00:00.0,2021-04-16T01:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D
1,2021-04-16T01:00:00.0,2021-04-16T02:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D
2,2021-04-16T02:00:00.0,2021-04-16T03:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D
3,2021-04-16T03:00:00.0,2021-04-16T04:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D
4,2021-04-16T04:00:00.0,2021-04-16T05:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D


In [67]:
df_soda.shape

(6888, 13)

The station list is loaded

In [68]:
df_estaciones = pd.read_csv(directorio + 'data/estaciones.csv')

In [69]:
df_estaciones.head()

,latitud,provincia,altitud,indicativo,nombre,longitud
0,413515N,BARCELONA,74,0252D,ARENYS DE MAR,023224E
1,411734N,BARCELONA,4,0076,BARCELONA AEROPUERTO,020412E
2,412506N,BARCELONA,408,0200E,"BARCELONA, FABRA",020727E
3,412326N,BARCELONA,6,0201D,BARCELONA,021200E
4,414312N,BARCELONA,291,0149X,MANRESA,015025E


Stations that are not in both data frames (stations and df_soda) are searched

In [70]:
estacion_quitar = []
# Stations in the station list
serie_indicativos = df_estaciones["indicativo"].unique().astype("str")
# Stations in the climate dataset
serie_estaciones = list(set(df_soda["estacion"].unique().astype("str")))

diferencia = len(serie_indicativos) - len(serie_estaciones)
print("The difference is: {}".format(diferencia))

# The callsigns of the stations in the list that are not in the dataset are saved
for i in range(0, len(serie_indicativos)):
    estacion = serie_indicativos[i]
    if estacion not in serie_estaciones:
        print("The number {} is missing, station {}".format(i, estacion))
        estacion_quitar.append(str(estacion))
        diferencia -= 1
        print("The difference is: {}".format(diferencia))
# The callsigns of the dataset stations that are not in the station list are saved
for i in range(0, len(serie_estaciones)):
    estacion = serie_estaciones[i]
    if (estacion not in list(serie_indicativos) and estacion not in estacion_quitar):
        print("The number {} is missing, station {}".format(i, estacion))
        estacion_quitar.append(str(estacion))
        diferencia += 1
        print("The difference is: {}".format(diferencia))
print(estacion_quitar)
print("The difference is: {}".format(diferencia))

The difference is: 0
The number 31 is missing, station 1249X
The difference is: -1
The number 124 is missing, station 1249I
The difference is: 0
['1249X', '1249I']
The difference is: 0


The list with the callsigns to be removed is displayed

In [71]:
estacion_quitar

['1249X', '1249I']

These stations are removed from the dataset

In [72]:
cont = 0
for i in range(0, len(df_soda["estacion"])):
    if str(df_soda["estacion"].loc[i]) in estacion_quitar:
        cont += 1
print("We have to remove {} rows from {}. {} will remain ".format(cont, len(df_soda["estacion"]), len(df_soda["estacion"])-cont))

We have to remove 24 rows from 6888. 6864 will remain 


A column is created that stores a *True* if the station in that row should be removed

In [73]:
estaciones_df = []

for i in range(0, len(df_soda["estacion"])):
    estacion = df_soda["estacion"].loc[i]
    if estacion in estacion_quitar:
        estaciones_df.append(True)
    else:
        estaciones_df.append(False)

In [74]:
df_soda.insert(len(df_soda.columns),"quitar",estaciones_df,True)

In [75]:
df_soda.head()

,dateBegins,dateEnds,toa,cs_ghi,cs_bhi,cs_dhi,cs_bni,ghi,bhi,dhi,bni,reliability,estacion,quitar
0,2021-04-16T00:00:00.0,2021-04-16T01:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,False
1,2021-04-16T01:00:00.0,2021-04-16T02:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,False
2,2021-04-16T02:00:00.0,2021-04-16T03:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,False
3,2021-04-16T03:00:00.0,2021-04-16T04:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,False
4,2021-04-16T04:00:00.0,2021-04-16T05:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,False


In [76]:
df_soda.drop(df_soda[df_soda["quitar"] == True].index, inplace = True)

In [77]:
df_soda.reset_index(drop=True, inplace=True)
df_soda.head()

,dateBegins,dateEnds,toa,cs_ghi,cs_bhi,cs_dhi,cs_bni,ghi,bhi,dhi,bni,reliability,estacion,quitar
0,2021-04-16T00:00:00.0,2021-04-16T01:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,False
1,2021-04-16T01:00:00.0,2021-04-16T02:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,False
2,2021-04-16T02:00:00.0,2021-04-16T03:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,False
3,2021-04-16T03:00:00.0,2021-04-16T04:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,False
4,2021-04-16T04:00:00.0,2021-04-16T05:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,False


In [78]:
print("Effectively, the new number of rows is {}".format(len(df_soda["estacion"])))

Effectively, the new number of rows is 6864


### Interesting columns are selected

We will need:
- ``dateBegins``: Date of the day to which the data correspond
- ``estacion``: Indicative code of the meteorological station

Columns ``bhi``, ``dhi``, ``bni``, ``toa``, ``reliability`` and ``dateEnds`` will not be necessary. The column ``remove`` and the columns of values for clear sky (cs) are also eliminated

In [79]:
df_soda.head()

,dateBegins,dateEnds,toa,cs_ghi,cs_bhi,cs_dhi,cs_bni,ghi,bhi,dhi,bni,reliability,estacion,quitar
0,2021-04-16T00:00:00.0,2021-04-16T01:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,False
1,2021-04-16T01:00:00.0,2021-04-16T02:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,False
2,2021-04-16T02:00:00.0,2021-04-16T03:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,False
3,2021-04-16T03:00:00.0,2021-04-16T04:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,False
4,2021-04-16T04:00:00.0,2021-04-16T05:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,False


In [80]:
df_soda.drop(['dateEnds'], axis=1, inplace = True)
df_soda.drop(['toa'], axis=1, inplace = True)
df_soda.drop(['cs_ghi'], axis=1, inplace = True)
df_soda.drop(['cs_bhi'], axis=1, inplace = True)
df_soda.drop(['cs_dhi'], axis=1, inplace = True)
df_soda.drop(['cs_bni'], axis=1, inplace = True)
df_soda.drop(['bhi'], axis=1, inplace = True)
df_soda.drop(['dhi'], axis=1, inplace = True)
df_soda.drop(['bni'], axis=1, inplace = True)
df_soda.drop(['reliability'], axis=1, inplace = True)
df_soda.drop(['quitar'], axis=1, inplace = True)
df_soda.head()

,dateBegins,ghi,estacion
0,2021-04-16T00:00:00.0,0.0,0252D
1,2021-04-16T01:00:00.0,0.0,0252D
2,2021-04-16T02:00:00.0,0.0,0252D
3,2021-04-16T03:00:00.0,0.0,0252D
4,2021-04-16T04:00:00.0,0.0,0252D


In [81]:
df_soda.shape

(6864, 3)

Using ``.info()`` NAs are seen.

In [82]:
df_soda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6864 entries, 0 to 6863
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   dateBegins  6864 non-null   object 
 1   ghi         6864 non-null   float64
 2   estacion    6864 non-null   object 
dtypes: float64(1), object(2)
memory usage: 161.0+ KB


The complete dataset is generated by concatenating all the saved daily files

In [83]:
fechas = []

# The first date is set (date of the first daily file)
now = datetime.now()
fecha_inicial = datetime(2021,4,5)

# The most current date is set
fecha_final = datetime(now.year,now.month,now.day)

# We get the list of days
rango_fechas = range((fecha_final - fecha_inicial).days + 1)
lista_fechas = [fechas.append("{}-{}-{}".format((fecha_inicial + timedelta(days = d)).year, "%02d" % (fecha_inicial + timedelta(days = d)).month, "%02d" % (fecha_inicial + timedelta(days = d)).day)) for d in rango_fechas] 

fechas

['2021-04-05',
 '2021-04-06',
 '2021-04-07',
 '2021-04-08',
 '2021-04-09',
 '2021-04-10',
 '2021-04-11',
 '2021-04-12',
 '2021-04-13',
 '2021-04-14',
 '2021-04-15',
 '2021-04-16',
 '2021-04-17',
 '2021-04-18',
 '2021-04-19',
 '2021-04-20',
 '2021-04-21',
 '2021-04-22',
 '2021-04-23',
 '2021-04-24',
 '2021-04-25',
 '2021-04-26',
 '2021-04-27',
 '2021-04-28',
 '2021-04-29',
 '2021-04-30',
 '2021-05-01',
 '2021-05-02',
 '2021-05-03',
 '2021-05-04',
 '2021-05-05',
 '2021-05-06',
 '2021-05-07',
 '2021-05-08',
 '2021-05-09',
 '2021-05-10',
 '2021-05-11',
 '2021-05-12',
 '2021-05-13',
 '2021-05-14',
 '2021-05-15',
 '2021-05-16',
 '2021-05-17',
 '2021-05-18',
 '2021-05-19',
 '2021-05-20',
 '2021-05-21',
 '2021-05-22',
 '2021-05-23',
 '2021-05-24',
 '2021-05-25',
 '2021-05-26',
 '2021-05-27',
 '2021-05-28',
 '2021-05-29',
 '2021-05-30',
 '2021-05-31',
 '2021-06-01',
 '2021-06-02',
 '2021-06-03',
 '2021-06-04',
 '2021-06-05',
 '2021-06-06',
 '2021-06-07',
 '2021-06-08',
 '2021-06-09',
 '2021-06-

In [84]:
df_soda_total = pd.DataFrame()

# For each daily file, the dataset is added
for date in fechas:
    try:
        df_soda = pd.read_csv(directorio + 'data/Rad_SODA/rad_soda_{}'.format(date))
        df_soda_total = df_soda_total.append(df_soda, ignore_index = True)
    except:
        continue
    
print(df_soda_total.shape)
df_soda_total.head()

(3417315, 13)


,dateBegins,dateEnds,toa,cs_ghi,cs_bhi,cs_dhi,cs_bni,ghi,bhi,dhi,bni,reliability,estacion
0,2021-04-17T21:00:00.0,2021-04-17T22:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D
1,2021-04-17T22:00:00.0,2021-04-17T23:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D
2,2021-04-17T23:00:00.0,2021-04-18T00:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D
3,2021-04-05T00:00:00.0,2021-04-05T01:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D
4,2021-04-05T01:00:00.0,2021-04-05T02:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D


### Management of NAs

The number of NAs per column and the % it represents of the total are shown

In [85]:
total = df_soda_total.isnull().sum().sort_values(ascending = False)
percent = (df_soda_total.isnull().sum() / df_soda_total.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
dhi,16764,0.004906
bhi,16764,0.004906
ghi,16764,0.004906
bni,16380,0.004793
estacion,0,0.000000
reliability,0,0.000000
cs_bni,0,0.000000
cs_dhi,0,0.000000
cs_bhi,0,0.000000
cs_ghi,0,0.000000


For columns with small NA %s, rows with missing data are eliminated

In [86]:
df_soda_total.dropna(inplace = True)
df_soda_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3400551 entries, 0 to 3417314
Data columns (total 13 columns):
 #   Column       Dtype  
---  ------       -----  
 0   dateBegins   object 
 1   dateEnds     object 
 2   toa          float64
 3   cs_ghi       float64
 4   cs_bhi       float64
 5   cs_dhi       float64
 6   cs_bni       float64
 7   ghi          float64
 8   bhi          float64
 9   dhi          float64
 10  bni          float64
 11  reliability  float64
 12  estacion     object 
dtypes: float64(10), object(3)
memory usage: 363.2+ MB


The hour column is obtained

In [87]:
df_soda_total['dateBegins'] = pd.to_datetime(df_soda_total['dateBegins'])
df_soda_total = df_soda_total.rename(columns={'dateBegins':'date'})
df_soda_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3400551 entries, 0 to 3417314
Data columns (total 13 columns):
 #   Column       Dtype         
---  ------       -----         
 0   date         datetime64[ns]
 1   dateEnds     object        
 2   toa          float64       
 3   cs_ghi       float64       
 4   cs_bhi       float64       
 5   cs_dhi       float64       
 6   cs_bni       float64       
 7   ghi          float64       
 8   bhi          float64       
 9   dhi          float64       
 10  bni          float64       
 11  reliability  float64       
 12  estacion     object        
dtypes: datetime64[ns](1), float64(10), object(2)
memory usage: 363.2+ MB


In [88]:
df_soda_total['hora'] = pd.to_datetime(df_soda_total['date']).dt.hour
df_soda_total['fecha'] = [str(a)[0:10] for a in df_soda_total['date']]

In [89]:
df_soda_total.head()

,date,dateEnds,toa,cs_ghi,cs_bhi,cs_dhi,cs_bni,ghi,bhi,dhi,bni,reliability,estacion,hora,fecha
0,2021-04-17 21:00:00,2021-04-17T22:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,21,2021-04-17
1,2021-04-17 22:00:00,2021-04-17T23:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,22,2021-04-17
2,2021-04-17 23:00:00,2021-04-18T00:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,23,2021-04-17
3,2021-04-05 00:00:00,2021-04-05T01:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,0,2021-04-05
4,2021-04-05 01:00:00,2021-04-05T02:00:00.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0252D,1,2021-04-05


In [90]:
df_soda_total = df_soda_total[(df_soda_total["hora"] < hora_fin) & (df_soda_total["hora"] >= hora_ini)]
df_soda_total.reset_index(drop=True, inplace=True)
df_soda_total.head()

,date,dateEnds,toa,cs_ghi,cs_bhi,cs_dhi,cs_bni,ghi,bhi,dhi,bni,reliability,estacion,hora,fecha
0,2021-04-05 05:00:00,2021-04-05T06:00:00.0,0.0,0.0,0.0,0.0,0.0,6.3289,0.4241,5.9048,7.6825,0.9667,0252D,5,2021-04-05
1,2021-04-05 06:00:00,2021-04-05T07:00:00.0,0.0,0.0,0.0,0.0,0.0,120.0964,57.4725,62.6239,268.0484,1.0000,0252D,6,2021-04-05
2,2021-04-05 07:00:00,2021-04-05T08:00:00.0,0.0,0.0,0.0,0.0,0.0,303.6657,194.8799,108.7858,509.9916,1.0000,0252D,7,2021-04-05
3,2021-04-05 08:00:00,2021-04-05T09:00:00.0,0.0,0.0,0.0,0.0,0.0,484.0124,348.3073,135.7052,640.0478,1.0000,0252D,8,2021-04-05
4,2021-04-05 09:00:00,2021-04-05T10:00:00.0,0.0,0.0,0.0,0.0,0.0,637.7261,486.6163,151.1098,720.5049,1.0000,0252D,9,2021-04-05


Possible repeated rows are eliminated

In [91]:
df_soda_total = df_soda_total.drop_duplicates(["date", 'fecha', 'hora', "estacion"],
                        keep = 'first')
df_soda_total.reset_index(drop = True, inplace = True)
df_soda_total.head()

,date,dateEnds,toa,cs_ghi,cs_bhi,cs_dhi,cs_bni,ghi,bhi,dhi,bni,reliability,estacion,hora,fecha
0,2021-04-05 05:00:00,2021-04-05T06:00:00.0,0.0,0.0,0.0,0.0,0.0,6.3289,0.4241,5.9048,7.6825,0.9667,0252D,5,2021-04-05
1,2021-04-05 06:00:00,2021-04-05T07:00:00.0,0.0,0.0,0.0,0.0,0.0,120.0964,57.4725,62.6239,268.0484,1.0000,0252D,6,2021-04-05
2,2021-04-05 07:00:00,2021-04-05T08:00:00.0,0.0,0.0,0.0,0.0,0.0,303.6657,194.8799,108.7858,509.9916,1.0000,0252D,7,2021-04-05
3,2021-04-05 08:00:00,2021-04-05T09:00:00.0,0.0,0.0,0.0,0.0,0.0,484.0124,348.3073,135.7052,640.0478,1.0000,0252D,8,2021-04-05
4,2021-04-05 09:00:00,2021-04-05T10:00:00.0,0.0,0.0,0.0,0.0,0.0,637.7261,486.6163,151.1098,720.5049,1.0000,0252D,9,2021-04-05


# Radiation final function
<div style = "float:right"><a style="text-decoration:none" href = "#Cleanup-and-preparation-of-historical-data-for-prediction-models">

In [92]:
def soda_clean(df_datos, df_puntos):
    
    # Stations that are not in both are deleted
    
    estacion_quitar = []
    # Stations in the station list
    serie_indicativos = df_estaciones["indicativo"].unique().astype("str")
    # Stations in the climate dataset
    serie_estaciones = list(set(df_datos["estacion"].unique().astype("str")))

    diferencia = len(serie_indicativos) - len(serie_estaciones)

    # The callsigns of the stations in the list that are not in the dataset are saved
    for i in range(0, len(serie_indicativos)):
        estacion = serie_indicativos[i]
        if estacion not in serie_estaciones:
            estacion_quitar.append(str(estacion))
            diferencia -= 1
    # The callsigns of the dataset stations that are not in the station list are saved
    for i in range(0, len(serie_estaciones)):
        estacion = serie_estaciones[i]
        if (estacion not in list(serie_indicativos) and estacion not in estacion_quitar):
            estacion_quitar.append(str(estacion))
            diferencia += 1
    
    estaciones_df = []
    
    for i in range(0, len(df_datos["estacion"])):
        estacion = df_datos["estacion"].loc[i]
        if estacion in estacion_quitar:
            estaciones_df.append(True)
        else:
            estaciones_df.append(False)
    df_datos.insert(len(df_datos.columns),"quitar",estaciones_df,True)
    df_datos.drop(df_datos[df_datos["quitar"] == True].index, inplace = True)
    df_datos.reset_index(drop=True, inplace=True)
    
    # Unnecessary columns are removed
    
    df_datos.drop(['dateEnds'], axis=1, inplace = True)
    df_datos.drop(['toa'], axis=1, inplace = True)
    df_datos.drop(['cs_ghi'], axis=1, inplace = True)
    df_datos.drop(['cs_bhi'], axis=1, inplace = True)
    df_datos.drop(['cs_dhi'], axis=1, inplace = True)
    df_datos.drop(['cs_bni'], axis=1, inplace = True)
    df_datos.drop(['bhi'], axis=1, inplace = True)
    df_datos.drop(['dhi'], axis=1, inplace = True)
    df_datos.drop(['bni'], axis=1, inplace = True)
    df_datos.drop(['reliability'], axis=1, inplace = True)
    df_datos.drop(['quitar'], axis=1, inplace = True)
    
    # Columns are converted to the correct data types
    
    df_datos['dateBegins'] = pd.to_datetime(df_datos['dateBegins'])
    df_datos = df_datos.rename(columns={'dateBegins':'date'})
    df_datos['hora'] = pd.to_datetime(df_datos['date']).dt.hour
    df_datos['fecha'] = [str(a)[0:10] for a in df_datos['date']]
    df_datos = df_datos[(df_datos["hora"] < hora_fin) & (df_datos["hora"] >= hora_ini)]
    df_datos.reset_index(drop = True, inplace = True)
    
    # Na's are eliminated
    
    df_datos.dropna(inplace = True)
    df_datos.reset_index(drop = True, inplace = True)
    
    # Possible repeated rows are eliminated
    
    df_datos = df_datos.drop_duplicates(["date", 'fecha', 'hora', "estacion"], keep = 'first')
    df_datos.reset_index(drop = True, inplace = True)
    df_datos.head()

    # Clean file is saved
    nombre = './data/Historicos/rad_soda_clean.csv'
    df_datos.to_csv(nombre, index = False)
    
    return df_datos

In [93]:
import numpy as np
import pandas as pd
import random
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import matplotlib.pyplot as plt
plt.style.use("seaborn")
from datetime import datetime, timedelta
hora_ini = 4
hora_fin = 20

# CSVs are read

fechas = []
now = datetime.now()
fecha_inicial = datetime(2021,4,5)
fecha_final = datetime(now.year,now.month,now.day)
rango_fechas = range((fecha_final - fecha_inicial).days + 1)
lista_fechas = [fechas.append("{}-{}-{}".format((fecha_inicial + timedelta(days = d)).year, "%02d" % (fecha_inicial + timedelta(days = d)).month, "%02d" % (fecha_inicial + timedelta(days = d)).day)) for d in rango_fechas] 

df_estaciones = pd.read_csv(directorio + 'data/estaciones.csv')

df_soda_total = pd.DataFrame()
for date in fechas:
    try:
        df_soda = pd.read_csv(directorio + 'data/Rad_SODA/rad_soda_{}'.format(date))
        df_soda_total = df_soda_total.append(df_soda, ignore_index = True)
    except:
        continue

# The function is called
df_clean = soda_clean(df_soda_total, df_estaciones)
    
df_clean.head()

,date,ghi,estacion,hora,fecha
0,2021-04-05 04:00:00,0.0000,0252D,4,2021-04-05
1,2021-04-05 05:00:00,6.3289,0252D,5,2021-04-05
2,2021-04-05 06:00:00,120.0964,0252D,6,2021-04-05
3,2021-04-05 07:00:00,303.6657,0252D,7,2021-04-05
4,2021-04-05 08:00:00,484.0124,0252D,8,2021-04-05


In [94]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345184 entries, 0 to 345183
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   date      345184 non-null  datetime64[ns]
 1   ghi       345184 non-null  float64       
 2   estacion  345184 non-null  object        
 3   hora      345184 non-null  int64         
 4   fecha     345184 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 13.2+ MB


### Sanity check

In [95]:
print('Different years: ', pd.to_datetime([year for year in pd.to_datetime(df_clean['fecha'])]).year.nunique())
print('Different months:', pd.to_datetime([year for year in pd.to_datetime(df_clean['fecha'])]).month.nunique())
print('There must be 31 different days:', pd.to_datetime([year for year in pd.to_datetime(df_clean['fecha'])]).day.nunique())


print('There should only be different hours within the filtering hours:', df_clean['hora'].nunique())

Different years:  1
Different months: 3
There must be 31 different days: 31
There should only be different hours within the filtering hours: 16


# Generation of the rows with the days in columns
<div style = "float:right"><a style="text-decoration:none" href = "#Cleanup-and-preparation-of-historical-data-for-prediction-models">

For each day that data is collected, there should only be one row per hour and station. For example, for historical weather data for the last 5 days, the data for each day must be columns associated with the hours of the day the data frame was downloaded (for each station, from 4 to 19, columns with the data from the previous day, columns with those of the previous one ...)

# Historical weather data

The csv is open

In [128]:
df_clima_total = pd.read_csv('./data/Historicos/historicos_climaticos_clean.csv', sep=',')
df_clima_total.head()

,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,clouds,visibility,wind_speed,wind_deg,wind_gust,we
0,2021-04-04,4,2021-04-05,0252D,284.30,282.56,1014,93,283.21,0,10000.0,2.57,280,4.689019,800
1,2021-04-04,5,2021-04-05,0252D,284.59,282.57,1014,93,283.50,0,10000.0,3.09,290,4.689019,800
2,2021-04-04,6,2021-04-05,0252D,284.41,283.88,1015,87,282.33,0,10000.0,0.51,0,4.689019,800
3,2021-04-04,7,2021-04-05,0252D,284.99,284.61,1016,87,282.90,0,10000.0,0.51,0,4.689019,800
4,2021-04-04,8,2021-04-05,0252D,286.41,286.17,1016,82,283.41,0,10000.0,0.51,0,4.689019,800


Rows are generated with every day of each call. The columns are labeled based on the day (d-1, d-2 ...)

In [129]:
columnas_1 = [col for col in df_clima_total.columns[1:4]] + [str(col+"_d-1") for col in df_clima_total.columns[4:]]
columnas_2 = [col for col in df_clima_total.columns[1:4]] + [str(col+"_d-2") for col in df_clima_total.columns[4:]]
columnas_3 = [col for col in df_clima_total.columns[1:4]] + [str(col+"_d-3") for col in df_clima_total.columns[4:]]
columnas_4 = [col for col in df_clima_total.columns[1:4]] + [str(col+"_d-4") for col in df_clima_total.columns[4:]]
columnas_5 = [col for col in df_clima_total.columns[1:4]] + [str(col+"_d-5") for col in df_clima_total.columns[4:]]

In [131]:
df_clima_dias_1 = pd.DataFrame(columns = columnas_1)
df_clima_dias_2 = pd.DataFrame(columns = columnas_2)
df_clima_dias_3 = pd.DataFrame(columns = columnas_3)
df_clima_dias_4 = pd.DataFrame(columns = columnas_4)
df_clima_dias_5 = pd.DataFrame(columns = columnas_5)

for i, fila in df_clima_total.iterrows():
    
    if (i in list(range(0, len(df_clima_total["date"]), 5000))) | (i == len(df_clima_total["date"])-1):
        print("Processing row {} of {}".format(i, len(df_clima_total["date"])))
        print("The number of rows in the datasets (approximately 1/5) is {}".format(len(df_clima_dias_1["hour"])))
        with open("./data/Historicos/Datasets_por_fecha/Registro_clima.txt",'a') as outFile:
            outFile.write('\n' + "Fila: " + str(i))
    
    # For each hourly row, we detect which day it belongs to and we attach it to the corresponding dataset
    if (pd.to_datetime(fila["fecha_prediccion"]) - pd.to_datetime(fila["date"])).days == 1:
        df_clima_dias_1.loc[len(df_clima_dias_1["fecha_prediccion"])] = [elem for elem in fila][1:]

    if (pd.to_datetime(fila["fecha_prediccion"]) - pd.to_datetime(fila["date"])).days == 2:
        df_clima_dias_2.loc[len(df_clima_dias_2["fecha_prediccion"])] = [elem for elem in fila][1:]
    
    if (pd.to_datetime(fila["fecha_prediccion"]) - pd.to_datetime(fila["date"])).days == 3:
        df_clima_dias_3.loc[len(df_clima_dias_3["fecha_prediccion"])] = [elem for elem in fila][1:]
        
    if (pd.to_datetime(fila["fecha_prediccion"]) - pd.to_datetime(fila["date"])).days == 4:
        df_clima_dias_4.loc[len(df_clima_dias_4["fecha_prediccion"])] = [elem for elem in fila][1:]
        
    if (pd.to_datetime(fila["fecha_prediccion"]) - pd.to_datetime(fila["date"])).days == 5:
        df_clima_dias_5.loc[len(df_clima_dias_5["fecha_prediccion"])] = [elem for elem in fila][1:]
        
    # Datatsets are saved periodically, in case memory becomes saturated 
    if (i in list(range(0, len(df_clima_total["date"]), 1000))) | (i == len(df_clima_total["date"])-1):
        
        # The file is saved
        nombre_1 = './data/Historicos/Datasets_por_fecha/clima_por_horas_1.csv'
        nombre_2 = './data/Historicos/Datasets_por_fecha/clima_por_horas_2.csv'
        nombre_3 = './data/Historicos/Datasets_por_fecha/clima_por_horas_3.csv'
        nombre_4 = './data/Historicos/Datasets_por_fecha/clima_por_horas_4.csv'
        nombre_5 = './data/Historicos/Datasets_por_fecha/clima_por_horas_5.csv'
        df_clima_dias_1.to_csv(nombre_1, index = False)
        df_clima_dias_2.to_csv(nombre_2, index = False)
        df_clima_dias_3.to_csv(nombre_3, index = False)
        df_clima_dias_4.to_csv(nombre_4, index = False)
        df_clima_dias_5.to_csv(nombre_5, index = False)
        

df_clima_dias_1.head()

Procesando fila 0 de 1870432
La cantidad de filas de los datasets (aproximadamente 1/5) es 0
Procesando fila 5000 de 1870432
La cantidad de filas de los datasets (aproximadamente 1/5) es 1008
Procesando fila 10000 de 1870432
La cantidad de filas de los datasets (aproximadamente 1/5) es 2000
Procesando fila 15000 de 1870432
La cantidad de filas de los datasets (aproximadamente 1/5) es 3008
Procesando fila 20000 de 1870432
La cantidad de filas de los datasets (aproximadamente 1/5) es 4000
Procesando fila 25000 de 1870432
La cantidad de filas de los datasets (aproximadamente 1/5) es 5008
Procesando fila 30000 de 1870432
La cantidad de filas de los datasets (aproximadamente 1/5) es 6000
Procesando fila 35000 de 1870432
La cantidad de filas de los datasets (aproximadamente 1/5) es 7008
Procesando fila 40000 de 1870432
La cantidad de filas de los datasets (aproximadamente 1/5) es 8000
Procesando fila 45000 de 1870432
La cantidad de filas de los datasets (aproximadamente 1/5) es 9008
Procesan

,hour,fecha_prediccion,estacion,temp_d-1,feels_like_d-1,pressure_d-1,humidity_d-1,dew_point_d-1,clouds_d-1,visibility_d-1,wind_speed_d-1,wind_deg_d-1,wind_gust_d-1,we_d-1
0,4,2021-04-05,0252D,284.30,282.56,1014,93,283.21,0,10000.0,2.57,280,4.689019,800
1,5,2021-04-05,0252D,284.59,282.57,1014,93,283.50,0,10000.0,3.09,290,4.689019,800
2,6,2021-04-05,0252D,284.41,283.88,1015,87,282.33,0,10000.0,0.51,0,4.689019,800
3,7,2021-04-05,0252D,284.99,284.61,1016,87,282.90,0,10000.0,0.51,0,4.689019,800
4,8,2021-04-05,0252D,286.41,286.17,1016,82,283.41,0,10000.0,0.51,0,4.689019,800


Saved datasets are loaded

In [132]:
df_clima_dias_1 = pd.read_csv('./data/Historicos/Datasets_por_fecha/clima_por_horas_1.csv', sep=',')

df_clima_dias_2 = pd.read_csv('./data/Historicos/Datasets_por_fecha/clima_por_horas_2.csv', sep=',')

df_clima_dias_3 = pd.read_csv('./data/Historicos/Datasets_por_fecha/clima_por_horas_3.csv', sep=',')

df_clima_dias_4 = pd.read_csv('./data/Historicos/Datasets_por_fecha/clima_por_horas_4.csv', sep=',')

df_clima_dias_5 = pd.read_csv('./data/Historicos/Datasets_por_fecha/clima_por_horas_5.csv', sep=',')
df_clima_dias_5.head()

,hour,fecha_prediccion,estacion,temp_d-5,feels_like_d-5,pressure_d-5,humidity_d-5,dew_point_d-5,clouds_d-5,visibility_d-5,wind_speed_d-5,wind_deg_d-5,wind_gust_d-5,we_d-5
0,4,2021-04-05,0252D,284.67,282.93,1026,68,278.98,42,9767.26843,1.12,338,1.18,802
1,5,2021-04-05,0252D,284.59,282.86,1026,67,278.69,50,9767.26843,1.02,336,1.10,800
2,6,2021-04-05,0252D,284.61,282.96,1026,68,278.92,41,9767.26843,0.97,324,1.07,802
3,7,2021-04-05,0252D,286.04,284.80,1026,64,279.40,47,9767.26843,0.54,329,0.85,802
4,8,2021-04-05,0252D,287.52,286.42,1026,58,279.37,54,9767.26843,0.32,121,0.67,803


In [133]:
print("Shape of day-1 dataset: {}".format(df_clima_dias_1.shape))
print("Shape of day-2 dataset: {}".format(df_clima_dias_2.shape))
print("Shape of day-3 dataset: {}".format(df_clima_dias_3.shape))
print("Shape of day-4 dataset: {}".format(df_clima_dias_4.shape))
print("Shape of day-5 dataset: {}".format(df_clima_dias_5.shape))

Shape del dataset día-1: (374192, 14)
Shape del dataset día-2: (374096, 14)
Shape del dataset día-3: (374064, 14)
Shape del dataset día-4: (374064, 14)
Shape del dataset día-5: (374016, 14)


The datasets are merged by prediction date, station and hour, to generate rows for each call day, hour and station

In [134]:
df_total = pd.merge(df_clima_dias_1, df_clima_dias_2, how = "inner", on = ["hour", "fecha_prediccion", "estacion"])
df_total = pd.merge(df_total, df_clima_dias_3, how = "inner", on = ["hour", "fecha_prediccion", "estacion"])
df_total = pd.merge(df_total, df_clima_dias_4, how = "inner", on = ["hour", "fecha_prediccion", "estacion"])
df_total = pd.merge(df_total, df_clima_dias_5, how = "inner", on = ["hour", "fecha_prediccion", "estacion"])
df_total.head()

,hour,fecha_prediccion,estacion,temp_d-1,feels_like_d-1,pressure_d-1,humidity_d-1,dew_point_d-1,clouds_d-1,visibility_d-1,wind_speed_d-1,wind_deg_d-1,wind_gust_d-1,we_d-1,temp_d-2,feels_like_d-2,pressure_d-2,humidity_d-2,dew_point_d-2,clouds_d-2,visibility_d-2,wind_speed_d-2,wind_deg_d-2,wind_gust_d-2,we_d-2,temp_d-3,feels_like_d-3,pressure_d-3,humidity_d-3,dew_point_d-3,clouds_d-3,visibility_d-3,wind_speed_d-3,wind_deg_d-3,wind_gust_d-3,we_d-3,temp_d-4,feels_like_d-4,pressure_d-4,humidity_d-4,dew_point_d-4,clouds_d-4,visibility_d-4,wind_speed_d-4,wind_deg_d-4,wind_gust_d-4,we_d-4,temp_d-5,feels_like_d-5,pressure_d-5,humidity_d-5,dew_point_d-5,clouds_d-5,visibility_d-5,wind_speed_d-5,wind_deg_d-5,wind_gust_d-5,we_d-5
0,4,2021-04-05,0252D,284.30,282.56,1014,93,283.21,0,10000.0,2.57,280,4.689019,800,285.17,283.96,1012,76,281.08,0,10000.0,1.03,280,4.689019,800,284.75,282.73,1014,76,280.67,0,10000.0,2.06,290,4.689019,800,282.28,280.06,1020,75,278.09,0,10000.0,1.54,310,4.689019,800,284.67,282.93,1026,68,278.98,42,9767.26843,1.12,338,1.18,802
1,5,2021-04-05,0252D,284.59,282.57,1014,93,283.50,0,10000.0,3.09,290,4.689019,800,284.66,283.21,1012,81,281.52,0,10000.0,1.54,320,4.689019,800,283.71,282.39,1014,81,280.59,0,10000.0,1.03,0,4.689019,800,281.30,279.00,1020,81,278.24,0,10000.0,1.70,288,1.760000,800,284.59,282.86,1026,67,278.69,50,9767.26843,1.02,336,1.10,800
2,6,2021-04-05,0252D,284.41,283.88,1015,87,282.33,0,10000.0,0.51,0,4.689019,800,285.18,284.34,1012,76,281.09,0,10000.0,0.51,0,4.689019,800,285.04,283.22,1014,71,279.96,0,10000.0,1.54,300,4.689019,800,281.42,279.47,1020,87,279.39,0,10000.0,1.54,280,4.689019,800,284.61,282.96,1026,68,278.92,41,9767.26843,0.97,324,1.07,802
3,7,2021-04-05,0252D,284.99,284.61,1016,87,282.90,0,10000.0,0.51,0,4.689019,800,287.22,286.25,1012,71,282.05,0,10000.0,1.03,0,4.689019,800,285.78,283.43,1015,71,280.67,0,10000.0,2.53,31,2.920000,800,284.12,282.78,1021,70,278.87,0,10000.0,0.51,0,4.689019,800,286.04,284.80,1026,64,279.40,47,9767.26843,0.54,329,0.85,802
4,8,2021-04-05,0252D,286.41,286.17,1016,82,283.41,0,10000.0,0.51,0,4.689019,800,288.86,287.74,1013,55,279.85,0,10000.0,0.51,0,4.689019,800,288.53,287.51,1015,58,280.32,0,10000.0,0.51,0,4.689019,800,286.82,285.25,1021,54,277.69,0,10000.0,0.51,150,4.689019,800,287.52,286.42,1026,58,279.37,54,9767.26843,0.32,121,0.67,803


In [135]:
nombre = './data/Historicos/clima_por_horas.csv'
df_total.to_csv(nombre, index = False)

# Weather prediction data

In [5]:
df_pred_total = pd.read_csv('./data/Historicos/predicciones_climaticas_clean.csv', sep=',')
df_pred_total.head()

,date,hour,fecha_prediccion,estacion,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,pop,we
0,2021-04-05,4,2021-04-05,0252D,285.47,284.66,1017,73,280.28,0.00,63,10000,0.98,306,1.36,0.0,803
1,2021-04-05,5,2021-04-05,0252D,285.54,284.66,1017,70,279.91,0.00,65,10000,1.18,10,1.43,0.0,803
2,2021-04-05,6,2021-04-05,0252D,285.31,284.44,1018,71,279.75,0.11,61,10000,1.73,34,1.71,0.0,803
3,2021-04-05,7,2021-04-05,0252D,286.09,285.19,1017,67,279.64,0.55,16,10000,1.52,58,1.76,0.0,801
4,2021-04-05,8,2021-04-05,0252D,287.24,286.32,1017,62,279.58,1.46,10,10000,0.94,103,1.28,0.0,800


Rows with every day of every call are generated. The days are classified into 2 groups. The first will contain the hours of the day that the data is obtained and of two days later. The second is the data for the day after the data is obtained. As the predictions for the 48 hours after the call are downloaded, these two groups will have 14 values each for each day and location (16 after filtering useful hours)

In [6]:
columnas_1 = [col for col in df_pred_total.columns[1:4]] + [str(col+"_pred_1") for col in df_pred_total.columns[4:]]
columnas_2 = [col for col in df_pred_total.columns[1:4]] + [str(col+"_pred_2") for col in df_pred_total.columns[4:]]

In [7]:
df_pred_dias_1 = pd.DataFrame(columns = columnas_1)
df_pred_dias_2 = pd.DataFrame(columns = columnas_2)

for i, fila in df_pred_total.iterrows():
    
    if (i in list(range(0,len(df_pred_total["date"]), 5000))) | (i == len(df_pred_total["date"])-1):
        print("Procesando fila {} de {}".format(i, len(df_pred_total["date"])))
        print("La cantidad de filas de los datasets (aproximadamente 1/2) es {}".format(len(df_pred_dias_1["hour"])))
        with open("./data/Historicos/Datasets_por_fecha/Registro_pred.txt",'a') as outFile:
            outFile.write('\n' + "Fila: " + str(i))
       
    # For each hourly row, we detect which day it belongs to and we attach it to the corresponding dataset
    if ((pd.to_datetime(fila["fecha_prediccion"]) - pd.to_datetime(fila["date"])).days == 0) | ((pd.to_datetime(fila["fecha_prediccion"]) - pd.to_datetime(fila["date"])).days == -2):
        df_pred_dias_1.loc[len(df_pred_dias_1["fecha_prediccion"])] = [elem for elem in fila][1:] 
        
    if (pd.to_datetime(fila["date"]) - pd.to_datetime(fila["fecha_prediccion"])).days == 1:
        df_pred_dias_2.loc[len(df_pred_dias_2["fecha_prediccion"])] = [elem for elem in fila][1:]

    # Datasets are saved periodically, in case memory saturates
    if (i in list(range(0,len(df_pred_total["date"]),1000))) | (i == len(df_pred_total["date"])-1):
        
        # The file is saved
        nombre_1 = './data/Historicos/Datasets_por_fecha/pred_por_horas_1.csv'
        nombre_2 = './data/Historicos/Datasets_por_fecha/pred_por_horas_2.csv'
        df_pred_dias_1.to_csv(nombre_1, index = False)
        df_pred_dias_2.to_csv(nombre_2, index = False)
        

df_pred_dias_2.head()

Procesando fila 0 de 768344
La cantidad de filas de los datasets (aproximadamente 1/2) es 0
Procesando fila 5000 de 768344
La cantidad de filas de los datasets (aproximadamente 1/2) es 2504
Procesando fila 10000 de 768344
La cantidad de filas de los datasets (aproximadamente 1/2) es 5824
Procesando fila 15000 de 768344
La cantidad de filas de los datasets (aproximadamente 1/2) es 8328
Procesando fila 20000 de 768344
La cantidad de filas de los datasets (aproximadamente 1/2) es 10832
Procesando fila 25000 de 768344
La cantidad de filas de los datasets (aproximadamente 1/2) es 13328
Procesando fila 30000 de 768344
La cantidad de filas de los datasets (aproximadamente 1/2) es 15824
Procesando fila 35000 de 768344
La cantidad de filas de los datasets (aproximadamente 1/2) es 18328
Procesando fila 40000 de 768344
La cantidad de filas de los datasets (aproximadamente 1/2) es 20832
Procesando fila 45000 de 768344
La cantidad de filas de los datasets (aproximadamente 1/2) es 23331
Procesando f

,hour,fecha_prediccion,estacion,temp_pred_2,feels_like_pred_2,pressure_pred_2,humidity_pred_2,dew_point_pred_2,uvi_pred_2,clouds_pred_2,visibility_pred_2,wind_speed_pred_2,wind_deg_pred_2,wind_gust_pred_2,pop_pred_2,we_pred_2
0,4,2021-04-05,0252D,284.28,283.56,1011,81,280.72,0.00,0,10000,4.15,31,5.29,0.0,800
1,5,2021-04-05,0252D,284.28,283.64,1011,84,281.19,0.00,3,10000,4.36,33,5.97,0.0,800
2,6,2021-04-05,0252D,284.93,284.36,1011,84,281.82,0.12,6,10000,4.17,46,6.10,0.0,800
3,7,2021-04-05,0252D,285.77,285.20,1012,81,282.26,0.38,49,10000,4.13,78,5.67,0.0,802
4,8,2021-04-05,0252D,285.76,285.19,1012,81,282.09,1.04,58,10000,3.40,90,4.50,0.0,803


Saved datasets are loaded

In [8]:
df_pred_dias_1 = pd.read_csv('./data/Historicos/Datasets_por_fecha/pred_por_horas_1.csv', sep=',')

df_pred_dias_2 = pd.read_csv('./data/Historicos/Datasets_por_fecha/pred_por_horas_2.csv', sep=',')
df_pred_dias_2.head()

,hour,fecha_prediccion,estacion,temp_pred_2,feels_like_pred_2,pressure_pred_2,humidity_pred_2,dew_point_pred_2,uvi_pred_2,clouds_pred_2,visibility_pred_2,wind_speed_pred_2,wind_deg_pred_2,wind_gust_pred_2,pop_pred_2,we_pred_2
0,4,2021-04-05,0252D,284.28,283.56,1011,81,280.72,0.00,0,10000,4.15,31,5.29,0.0,800
1,5,2021-04-05,0252D,284.28,283.64,1011,84,281.19,0.00,3,10000,4.36,33,5.97,0.0,800
2,6,2021-04-05,0252D,284.93,284.36,1011,84,281.82,0.12,6,10000,4.17,46,6.10,0.0,800
3,7,2021-04-05,0252D,285.77,285.20,1012,81,282.26,0.38,49,10000,4.13,78,5.67,0.0,802
4,8,2021-04-05,0252D,285.76,285.19,1012,81,282.09,1.04,58,10000,3.40,90,4.50,0.0,803


In [9]:
print("Shape of the day/day+2 dataset: {}".format(df_pred_dias_1.shape))
print("Shape of the day+1 dataset: {}".format(df_pred_dias_2.shape))

Shape del dataset día/dia+2: (385864, 16)
Shape del dataset día+1: (382480, 16)


The datasets are merged by prediction date, station and hour, to generate rows for each call day, hour and station

In [10]:
# Possible duplicate rows are eliminated

df_pred_dias_1 = df_pred_dias_1.drop_duplicates(['hour', "fecha_prediccion", "estacion"],
                        keep = 'first')
df_pred_dias_1.reset_index(drop = True, inplace = True)
df_pred_dias_2 = df_pred_dias_2.drop_duplicates(['hour', "fecha_prediccion", "estacion"],
                        keep = 'first')
df_pred_dias_2.reset_index(drop = True, inplace = True)

# Datasets are joined
df_total_previo = pd.merge(df_pred_dias_1, df_pred_dias_2, how = "inner", on = ["hour", "fecha_prediccion", "estacion"])
df_total_previo.head()

,hour,fecha_prediccion,estacion,temp_pred_1,feels_like_pred_1,pressure_pred_1,humidity_pred_1,dew_point_pred_1,uvi_pred_1,clouds_pred_1,visibility_pred_1,wind_speed_pred_1,wind_deg_pred_1,wind_gust_pred_1,pop_pred_1,we_pred_1,temp_pred_2,feels_like_pred_2,pressure_pred_2,humidity_pred_2,dew_point_pred_2,uvi_pred_2,clouds_pred_2,visibility_pred_2,wind_speed_pred_2,wind_deg_pred_2,wind_gust_pred_2,pop_pred_2,we_pred_2
0,4,2021-04-05,0252D,285.47,284.66,1017,73,280.28,0.00,63,10000,0.98,306,1.36,0.0,803,284.28,283.56,1011,81,280.72,0.00,0,10000,4.15,31,5.29,0.0,800
1,5,2021-04-05,0252D,285.54,284.66,1017,70,279.91,0.00,65,10000,1.18,10,1.43,0.0,803,284.28,283.64,1011,84,281.19,0.00,3,10000,4.36,33,5.97,0.0,800
2,6,2021-04-05,0252D,285.31,284.44,1018,71,279.75,0.11,61,10000,1.73,34,1.71,0.0,803,284.93,284.36,1011,84,281.82,0.12,6,10000,4.17,46,6.10,0.0,800
3,7,2021-04-05,0252D,286.09,285.19,1017,67,279.64,0.55,16,10000,1.52,58,1.76,0.0,801,285.77,285.20,1012,81,282.26,0.38,49,10000,4.13,78,5.67,0.0,802
4,8,2021-04-05,0252D,287.24,286.32,1017,62,279.58,1.46,10,10000,0.94,103,1.28,0.0,800,285.76,285.19,1012,81,282.09,1.04,58,10000,3.40,90,4.50,0.0,803


For each day and station, we obtain 2 prediction values for each hour (two for 4 o'clock, two for 5 o'clock ...) corresponding to the prediction of the 48 hours following the call. Then we generate the dataset with the average prediction value of each hour

In [11]:
columnas_total = [col for col in df_pred_total.columns[1:4]] + [str(col+"_pred") for col in df_pred_total.columns[4:]]
df_total = pd.DataFrame(columns = columnas_total)

for i, fila in df_total_previo.iterrows():
    
    if (i in list(range(0,len(df_total_previo["hour"]),5000))) | (i == len(df_total_previo["hour"])-1):
        print("Procesando fila {} de {}".format(i, len(df_total_previo["hour"])))
       
    # For each hour, we obtain the mean of the data of the two predicted values
    fila_nueva = []
    df_new = pd.DataFrame()
    for j in range(0, len(columnas_total)):
        if j in [0,1,2]:
            fila_nueva.append(fila[j]) 
        else:
            fila_nueva.append(np.mean([fila[j], fila[j + (int((len(columnas_total)-3)))]]))
    df_new = pd.DataFrame([tuple(fila_nueva)], columns = columnas_total)
    df_total = df_total.append(df_new, ignore_index = True)
    
    
df_total.head()

Procesando fila 0 de 382480
Procesando fila 5000 de 382480
Procesando fila 10000 de 382480
Procesando fila 15000 de 382480
Procesando fila 20000 de 382480
Procesando fila 25000 de 382480
Procesando fila 30000 de 382480
Procesando fila 35000 de 382480
Procesando fila 40000 de 382480
Procesando fila 45000 de 382480
Procesando fila 50000 de 382480
Procesando fila 55000 de 382480
Procesando fila 60000 de 382480
Procesando fila 65000 de 382480
Procesando fila 70000 de 382480
Procesando fila 75000 de 382480
Procesando fila 80000 de 382480
Procesando fila 85000 de 382480
Procesando fila 90000 de 382480
Procesando fila 95000 de 382480
Procesando fila 100000 de 382480
Procesando fila 105000 de 382480
Procesando fila 110000 de 382480
Procesando fila 115000 de 382480
Procesando fila 120000 de 382480
Procesando fila 125000 de 382480
Procesando fila 130000 de 382480
Procesando fila 135000 de 382480
Procesando fila 140000 de 382480
Procesando fila 145000 de 382480
Procesando fila 150000 de 382480
Pr

,hour,fecha_prediccion,estacion,temp_pred,feels_like_pred,pressure_pred,humidity_pred,dew_point_pred,uvi_pred,clouds_pred,visibility_pred,wind_speed_pred,wind_deg_pred,wind_gust_pred,pop_pred,we_pred
0,4,2021-04-05,0252D,284.875,284.110,1014.0,77.0,280.500,0.000,31.5,10000.0,2.565,168.5,3.325,0.0,801.5
1,5,2021-04-05,0252D,284.910,284.150,1014.0,77.0,280.550,0.000,34.0,10000.0,2.770,21.5,3.700,0.0,801.5
2,6,2021-04-05,0252D,285.120,284.400,1014.5,77.5,280.785,0.115,33.5,10000.0,2.950,40.0,3.905,0.0,801.5
3,7,2021-04-05,0252D,285.930,285.195,1014.5,74.0,280.950,0.465,32.5,10000.0,2.825,68.0,3.715,0.0,801.5
4,8,2021-04-05,0252D,286.500,285.755,1014.5,71.5,280.835,1.250,34.0,10000.0,2.170,96.5,2.890,0.0,801.5


In [12]:
nombre = './data/Historicos/pred_por_horas.csv'
df_total.to_csv(nombre, index = False)